In [1]:
# Washington Multiple AGN (WMAGN) Catalog DR1 - Quasar Pair Catalog Formatting
# Author: R. W. Pfeifle
# Legacy Notebook Created: 10 Sept. 2020
# Quasar Pairs Notebook Created: 28 March 2023
# Last Revision Date: 28 March 2023

# Purpose:
# 
# 

Update 16 Sept. 2020
---


Here is how I want to construct the catalog now and moving forward:

- There will be TWO catalogs, which are permutations of a single list. 
    - One catalog will have only one system per line, and we will have ONLY pairs in this one. This will make calculations FAR easier for individual systems (such as redshift or velocity differences), as this circumvents the problems of not being able to do row-wise operations.
    - The other catalog will be a copy of the first, with the exception that this catalog will have one line per system component, i.e. two lines per pair. This will be the catalog used for matching to other catalogs. Thus this one is a derivative of the first.
    - This same setup will be used for systems of triples, quadruples, quintuples, etc. Different order multiplets will be in different catalogs. This will make things easier to comprehend and will eliminate needed a million columns within only a single table that are empty for any system with fewer than the max number. 


--------
Summary:
--------
The purpose of this catalog is to have an exhaustive list of all known multiple
AGNs from the literature. A multiple AGN is defined as the manifestation of two
or more AGNs at some wavelength that are associated with each other due to some
astrophysical process. This astrophysical process can be:

- Multiple AGNs in a galaxy interaction, merger, or post-merger
- Multiple AGNs that are gravitationally bound, e.g. binary AGNs
- The appearance of multiple AGNs due to gravitational lensing (e.g. an Einstein cross)

This catalog should be *exhaustive*, so include not only confirmed systems but
also candidate systems. These systems should be everything that indicates or
potentially indicates the presence or apparent presence of more than one AGN
associated with a system (e.g. multiple X-ray/radio component, doubling of
spectroscopic lines, astrometric jitter, etc.)

----------------------
WMAGN catalog columns:
----------------------

- System type (Dual/triple AGN, binary AGN, or gravitational lens)

- Discovery method (e.g., multiple X-ray/radio components, spectroscopic, astrometric, etc.)

- Literature name (e.g. SDSS J084905.51+111447.2)

- Component (e.g. Gal1/Gal2/Gal3, a/b/c, 1/2/3, etc)
- One row for each component; components share same unique system name

- Unique, standardized system name
    - NED, Simbad, or IAU name preferred

- Coordinates
    - Must be in ICRS system. If listed in FK5/FK6, convert to ICRS (e.g. with AstroPy)
    - Equinox J2000.0
    - Native epoch (i.e. not corrected for "proper motion")
    - Degrees, full precision (i.e. 0.00001 deg = 0.036 arcsec)
    - If coordinates not available for each component individually, just repeat the listed system coordinates for each source.

- Coordinate epoch (e.g. 2015.5 for Gaia DR2)
    - Leave blank if not determinable

- Coordinate wavelength/frequency
    - Leave blank if not determinable

- Coordinate source (e.g. SDSS, 2MASS, VLBA, Chandra, etc)

- Redshift
    - Leave blank if not determinable

- Redshift type (spec/photo)
    - Leave blank if not determinable

- Brightness band
    - E.g., Johnson V, SDSS r, Gaia G, WISE W1, radio X-band, etc.

- Brightness numerical value
    - Magnitude or flux density

- Brightness type
    - E.g., VEGAMAG, ABMAG/STMAG, Jy

- Bibcode(s), e.g. 2019ApJ...883..167P,2019ApJ...887...90L
    - Ordered by date

- NED hyperlink

- Simbad hyperlink

- Confidence level
    - Up to you how you want to qualify this. E.g., "potential", "likely", "confirmed"
    - Preferably numeric so catalog can be sorted. E.g. 1=potential, 2=likely, 3=confirmed.

- Essential object notes


In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
#from astroquery.simbad import Simbad
#from astroquery.sdss import SDSS

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)

In [3]:
# This function is for converting namings in the following format to coordinates. This is useful for later \
# catalogs 2010-2020

def name_to_coords(df,dfcol):
    # A function to take in the SDSS designation in a form JXXXXXX.XX+XXXXXX.X and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
    # Output form will be: XX:XX:XX.XX +XX:XX:XX.X, where the '+' could be '+' or '-'
    df['Coordinates'] = dfcol.str.slice(start=1) # Stripping the J
    df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
    df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
    df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
    df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
    df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
    return


In [6]:
def name_to_coords(df):
    # A function to take in the SDSS designation in a form JXXXXXX.XX+XXXXXX.X and convert this to sexagesimal coordinates, where the '+' could be '+' or '-'.
    # Output form will be: XX:XX:XX.XX +XX:XX:XX.X, where the '+' could be '+' or '-'
    df['Coordinates'] = df['SDSS'].str.slice(start=1) # Stripping the J
    df['RA_test'] = df['Coordinates'].str.slice(start=0, stop=9) # Stripping the DEC parts 
    df['Dec_test'] = df['Coordinates'].str.slice(start=9, stop=19) # Stripping the RA parts
    df['RA'] = df['RA_test'].str.slice(start=0, stop=2)+":"+df['RA_test'].str.slice(start=2, stop=4)+":"+df['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
    df['Dec'] = df['Dec_test'].str.slice(start=0, stop=3)+":"+df['Dec_test'].str.slice(start=3, stop=5)+":"+df['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
    df.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)
    return


In [7]:
# Here we're loading in the catalog(s) from Hennawi+2006:

# Table 2 are binary quasars discovered from follow-up observations (sep 3 arcsec < delta theta < 60 arcsec)
hennawi2006_t2 = (Table.read('Hennawi2006/table2.dat', readme = 'Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()

# Table 3 are binary quasars discovered via the lens selection (sep <3 arcsec)
hennawi2006_t3 = (Table.read('Hennawi2006/table3.dat', readme = 'Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()
hennawi2006_t3['Sample'] = 'Lens Selection'

# Table 4 are binary quasars discovered overlapping plates from SDSS + 2DQZ quasars
hennawi2006_t4 = (Table.read('Hennawi2006/table4.dat', readme = 'Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()
hennawi2006_t4['Sample'] = 'Overlapping Plates'

# Table 8 contains projected quasar pairs discovered via overlapping plates and the SDSS +2QZ catalog. 
# Some of these are not actually projected pairs (after inspection of the table), and therefore we will need to \
# include this table (and cut it down to only the relevant listings).
hennawi2006_t8 = (Table.read('Hennawi2006/table8.dat', readme = 'Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()
hennawi2006_t8['dV'] = 'None'
# Need to overwrite the NaN values in the 'Sample' column so that it reflects the fact that these come from \
# Table 8 'Project Pairs'
hennawi2006_t8['Sample'] = "Projected Pairs"

# Table 9 contains projected pairs discovered via overlapping plates. No need to include in catalog or load in here.
#hennawi2006_t9 = (Table.read('Hennawi2006/table9.dat', readme = 'Hennawi2006/ReadMe', format='ascii.cds')).to_pandas()

# Table 10 has only quasar-star pairs. No need to include in catalog or load in here.


# MAGNITUDES QUOTED IN HENNAWI+ TABLES ARE EXTINCTION CORRECTED!

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3331: TableReplaceWarning: converted column 'dV' from integer to float
  exec(code_obj, self.user_global_ns, self.user_ns)


In [8]:
#hennawi2006_t3

In [9]:
# All of the Hennawi+2006 catalogs are structured the same, so we may as well combine them all here, and then \
# perform any column operations or column renaming that is required.

# Now concatenating Tables 2, 3 and 4 of Hennawi+2006 catalogs 
frames = [hennawi2006_t2,hennawi2006_t3,hennawi2006_t4,hennawi2006_t8]
hennawi2006 = (pd.concat(frames)).reset_index(drop=True) # Now we have a master catalog for the Hennawi+ tables.

# Now creating appropriate RA and Dec coordinate columns:
hennawi2006['RA1'] = hennawi2006['RA1.h'].astype(str) + ':' + hennawi2006['RA1.m'].astype(str) + ':' + hennawi2006['RA1.s'].astype(str)
hennawi2006['Dec1'] = hennawi2006['DE1.-'].astype(str) + hennawi2006['DE1.d'].astype(str) + ':' + hennawi2006['DE1.m'].astype(str) + ':' + hennawi2006['DE1.s'].astype(str)

hennawi2006['RA2'] = hennawi2006['RA2.h'].astype(str) + ':' + hennawi2006['RA2.m'].astype(str) + ':' + hennawi2006['RA2.s'].astype(str)
hennawi2006['Dec2'] = hennawi2006['DE2.-'].astype(str) + hennawi2006['DE2.d'].astype(str) + ':' + hennawi2006['DE2.m'].astype(str) + ':' + hennawi2006['DE2.s'].astype(str)

# Now dropping the original 14 columns used for RA and Dec...
hennawi2006.drop(labels=['RA1.h','RA1.m','RA1.s','DE1.-','DE1.d','DE1.m','DE1.s','RA2.h','RA2.m','RA2.s','DE2.-','DE2.d','DE2.m','DE2.s'], axis=1, inplace=True)

# Dropping irrelevant magnitude columns (we likely won't be using these anyhow...)
hennawi2006.drop(labels=['umag1','rmag1','imag1','zmag1','umag2','rmag2','imag2','zmag2'], axis=1, inplace=True)

# Substituting in information for SDSS J1116+4118A/B from Ellison+2007 here:
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'z1'] = 2.982
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'z2'] = 3.007

# Adding informative columns about the coordinates
hennawi2006['Equinox'] = "J2000"
hennawi2006['Coordinate_waveband'] = "Optical"
hennawi2006['Coordinate_Source'] = "SDSS"

hennawi2006['System Type'] = 'Binary Quasar'
hennawi2006['z1_type'] = 'spec'
hennawi2006['z2_type'] = 'spec'

hennawi2006['Brightness_band1'] = 'SDSS g PSF ext-corr'
hennawi2006['Brightness_band2'] = 'SDSS g PSF ext-corr'

hennawi2006['Brightness_type1'] = 'asinh'
hennawi2006['Brightness_type2'] = 'asinh'

# Calculating here redshift difference (delta_z) and velocity differences dV
# dV is defined as: dv = (2.99e+5)*((1+z1)**2 - (1+z2)**2)/((1+z1)**2+(1+z2)**2)
# These come from  the work in Tytler+2009 for velocity differences and LOS distances derived from velocity differences

#v=H(z)*d, [H(z)/H0]**2 = ((1+z)**3)*Om0 + OmV
#    Therefore: H(z) = H0*np.sqrt(((1+z)**3)*Om0 + OmV)
#d = v/H(z)

hennawi2006['delta_z'] = hennawi2006['z1'] - hennawi2006['z2']
hennawi2006['dV_rwp'] = (2.99e+5)*((1+hennawi2006['z1'])**2 - (1+hennawi2006['z2'])**2)/((1+hennawi2006['z1'])**2+(1+hennawi2006['z2'])**2)
# Defining it as dV_rwp for now since Hennawi+ also calculated this, and the values are discrepant with mine.

# The following will also calculate the LOS distance between the two objects
#hennawi2006['LOS_dist(Mpc)'] = np.abs(hennawi2006['dV_new'])/(70*np.sqrt(((1+hennawi2006['z1'])**3)*0.3 + 0.7))

# Now calculating the projected separation in kpc from the arcsec separation...
#hennawi2006['r_kpc'] = hennawi2006['r']/(0.72) # This was a check on Hennawi's calculations...
hennawi2006['Sep(kpc)'] = hennawi2006['theta']*((cosmo.arcsec_per_kpc_proper(hennawi2006['z1']))**(-1))
# The separation is based upon the redshift of the first quasar in each pair, but the difference should be \
# minimal if z2 was used instead.

# Now adding in a few columns to denote the confirmation method, author name+year, bibcode, and doi link
hennawi2006['Selection Method'] = hennawi2006['Sample']
hennawi2006['Confirmation Method'] = "Optical Spectroscopy"
hennawi2006['Paper(s)'] = "Hennawi+2006"
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'Paper(s)'] = "Hennawi+2006 ; Ellison+2007"
hennawi2006['BibCode(s)'] = "2006AJ....131....1H"
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'Bibcode(s)'] = "2006AJ....131....1H ; 2007MNRAS.378..801E "
hennawi2006['DOI(s)'] = "https://doi.org/10.1086/498235"
hennawi2006.loc[hennawi2006.Name1 == 'SDSSJ1116+4118A', 'DOI(s)'] = "https://doi.org/10.1086/498235 ; https://doi.org/10.1111/j.1365-2966.2007.11847.x"

# Taking into account the overlap between Hennawi+2006 and Miller+2004
hennawi2006.loc[hennawi2006.Name1 == '2QZJ1435+0008A', 'Paper(s)'] = "Miller+2004 ; Hennawi+2006"
hennawi2006.loc[hennawi2006.Name1 == '2QZJ1435+0008A', 'Bibcode(s)'] = "2004MNRAS.348..395M ; 2006AJ....131....1H"
hennawi2006.loc[hennawi2006.Name1 == '2QZJ1435+0008A', 'DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2004.07303.x ; https://doi.org/10.1086/498235"

# Taking into account the overlap between Pindor+2006 and Hennawi+2006: Pindor+2006 identified the dual quasar \
# SDSSJ1600+0000A. (Pindor is also an author on Hennawi's paper)
hennawi2006.loc[hennawi2006.Name1=='SDSSJ1600+0000A', 'Paper(s)'] = "Pindor+2006 ; Hennawi+2006"
hennawi2006.loc[hennawi2006.Name1=='SDSSJ1600+0000A', 'BibCode(s)'] = "2006AJ....131...41P ; 2006AJ....131....1H"
hennawi2006.loc[hennawi2006.Name1=='SDSSJ1600+0000A', 'DOI(s)'] = "https://doi.org/10.1086/497965 ; https://doi.org/10.1086/498235"

hennawi2006['PairID'] = hennawi2006.index

# Now removing all systems which do not meet our cut in velocity difference of ≤2000 km s**-1
hennawi2006 = (hennawi2006[np.abs(hennawi2006['dV_rwp'])<=2000]).reset_index(drop=True)
# (This leaves 284 pairs)

# Here we're dropping the obvious duplicates where the A and B components are not mixed up between tables
hennawi2006 = hennawi2006.drop_duplicates(subset=['Name1','Name2','z1','z2','theta','RA1','Dec1','RA2','Dec2'])
hennawi2006 = hennawi2006.drop_duplicates(subset=['Name1','Name2']) #removing duplicates that have correct components only

# Here we need to do something clever, because Hennawi+2006 mixed up the A and B designations between different \
# tables I'm going to make two new columns: sum of z's and sum of gmag's
hennawi2006['summedz'] = hennawi2006['z1']+hennawi2006['z2']
hennawi2006['summedgmag'] = hennawi2006['gmag1']+hennawi2006['gmag2']

# Now this allows me to circumvent the issue of mixed up redshift and gmag columns. Nothing I can do about the \
# mixed up names, but NONE of the rows should have duplicate summed redshifts AND gmags AND identical separations \
# unless the rows are in fact the same pair.
hennawi2006 = hennawi2006.drop_duplicates(subset=['summedz','summedgmag','theta'])

# Now dropping any remaining irrelevant columns:
hennawi2006.drop(labels=['Sample','dV','chi2','PairID','summedz','summedgmag'], axis=1, inplace=True)

coordconvert = SkyCoord(ra = hennawi2006['RA1'], dec = hennawi2006['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2006['RA1_deg'] = coordconvert.ra.degree
hennawi2006['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = hennawi2006['RA2'], dec = hennawi2006['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2006['RA2_deg'] = coordconvert.ra.degree
hennawi2006['Dec2_deg'] = coordconvert.dec.degree

hennawi2006.reset_index(drop=True, inplace=True)
               

In [10]:
hennawi2006 = hennawi2006[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type',\
                           'z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
                           'Equinox','Coordinate_waveband','Coordinate_Source','gmag1','Brightness_band1',\
                           'Brightness_type1','gmag2','Brightness_band2','Brightness_type2','theta','Sep(kpc)','r','delta_z',\
                           'dV_rwp','Paper(s)','BibCode(s)','DOI(s)']]

hennawi2006.drop(labels=['r'], axis=1, inplace=True) # Dropping this column, since we separately calculate it in 'Sep(kpc)'

# Here we're renaming the columns:
# dV_rwp --> dV
# gmag1 --> Brightness1 and gmag2 --> Brightness2
hennawi2006.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type',\
                           'z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
                           'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
                           'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep',\
                           'Sep(kpc)','delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']


In [11]:
#hennawi2006

In [5]:
hennawi2006.to_csv('Hennawi2006_formattedwmagn.csv', sep=',')


In [17]:
# Here we're loading in the Hennawi+2010 tables:

# Table 2 are binary quasars discovered from follow-up observations (sep 3 arcsec < delta theta < 60 arcsec)
hennawi2010 = (Table.read('Tables/Hennawi2010/table3.dat', readme = 'Tables/Hennawi2010/ReadMe', format='ascii.cds')).to_pandas()

#hennawi2010

primary_df = df.iloc[::2].reset_index(drop=True)
secondary_df = df.iloc[1::2].reset_index(drop=True)

# Append a suffix to the column names of primary and secondary DataFrames
primary_df.columns = [f"{col}_1" for col in primary_df.columns]
secondary_df.columns = [f"{col}_2" for col in secondary_df.columns]

# Concatenate the primary and secondary DataFrames horizontally
hennawi2010 = pd.concat([primary_df, secondary_df], axis=1)


# Now creating appropriate RA and Dec coordinate columns:
hennawi2010['RA1'] = hennawi2010['RAh_1'].astype(str) + ':' + hennawi2010['RAm_1'].astype(str) + ':' + hennawi2010['RAs_1'].astype(str)
hennawi2010['Dec1'] = hennawi2010['DE-_1'].astype(str) + hennawi2010['DEd_1'].astype(str) + ':' + hennawi2010['DEm_1'].astype(str) + ':' + hennawi2010['DEs_2'].astype(str)

hennawi2010['RA2'] = hennawi2010['RAh_2'].astype(str) + ':' + hennawi2010['RAm_2'].astype(str) + ':' + hennawi2010['RAs_2'].astype(str)
hennawi2010['Dec2'] = hennawi2010['DE-_2'].astype(str) + hennawi2010['DEd_2'].astype(str) + ':' + hennawi2010['DEm_2'].astype(str) + ':' + hennawi2010['DEs_2'].astype(str)


# Now dropping the original 14 columns used for RA and Dec...
hennawi2010.drop(labels=['RAh_1','RAm_1','RAs_1','DE-_1','DEd_1','DEm_1','DEs_1', 'RAh_2','RAm_2','RAs_2','DE-_2','DEd_2','DEm_2','DEs_2','f_[HMS2010]_1','f_[HMS2010]_2'], axis=1, inplace=True)

hennawi2010.columns = hennawi2010.columns.str.replace('_', '')

# Dropping irrelevant magnitude columns (we likely won't be using these anyhow...)
hennawi2010.drop(labels=['umag1','urmag1','rmag1','imag1','zmag1','umag2','rmag2','urmag2','imag2','zmag2'], axis=1, inplace=True)

# Substituting in information for SDSS J1116+4118A/B from Ellison+2007 here:
#hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'z1'] = 2.982
#hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'z2'] = 3.007

# Adding informative columns about the coordinates
hennawi2010['Equinox'] = "J2000"
hennawi2010['Coordinate_waveband'] = "Optical"
hennawi2010['Coordinate_Source'] = "SDSS"

hennawi2010['System Type'] = 'Binary Quasar'
hennawi2010['z1_type'] = 'spec'
hennawi2010['z2_type'] = 'spec'

hennawi2010['Brightness_band1'] = 'SDSS g PSF ext-corr'
hennawi2010['Brightness_band2'] = 'SDSS g PSF ext-corr'

hennawi2010['Brightness_type1'] = 'asinh'
hennawi2010['Brightness_type2'] = 'asinh'

# Calculating here redshift difference (delta_z) and velocity differences dV
# dV is defined as: dv = (2.99e+5)*((1+z1)**2 - (1+z2)**2)/((1+z1)**2+(1+z2)**2)
# These come from  the work in Tytler+2009 for velocity differences and LOS distances derived from velocity differences

#v=H(z)*d, [H(z)/H0]**2 = ((1+z)**3)*Om0 + OmV
#    Therefore: H(z) = H0*np.sqrt(((1+z)**3)*Om0 + OmV)
#d = v/H(z)

#hennawi2010['delta_z'] = hennawi2010['z1'] - hennawi2010['z2']
#hennawi2010['dV_rwp'] = (2.99e+5)*((1+hennawi2010['z1'])**2 - (1+hennawi2010['z2'])**2)/((1+hennawi2010['z1'])**2+(1+hennawi2010['z2'])**2)
# Defining it as dV_rwp for now since Hennawi+ also calculated this, and the values are discrepant with mine.

# The following will also calculate the LOS distance between the two objects
#hennawi2010['LOS_dist(Mpc)'] = np.abs(hennawi2010['dV_new'])/(70*np.sqrt(((1+hennawi2010['z1'])**3)*0.3 + 0.7))

# Now calculating the projected separation in kpc from the arcsec separation...
#hennawi2010['r_kpc'] = hennawi2010['r']/(0.72) # This was a check on Hennawi's calculations...
#hennawi2010['Sep(kpc)'] = hennawi2010['theta']*((cosmo.arcsec_per_kpc_proper(hennawi2010['z1']))**(-1))
# The separation is based upon the redshift of the first quasar in each pair, but the difference should be \
# minimal if z2 was used instead.

# Now adding in a few columns to denote the confirmation method, author name+year, bibcode, and doi link
hennawi2010['Selection Method'] = ' '
hennawi2010['Confirmation Method'] = "Optical Spectroscopy"
hennawi2010['Paper(s)'] = "Hennawi+2010"
#hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'Paper(s)'] = "Hennawi+2006 ; Ellison+2007"
hennawi2010['BibCode(s)'] = "2010ApJ...719.1672H"
#hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'Bibcode(s)'] = "2006AJ....131....1H ; 2007MNRAS.378..801E "
hennawi2010['DOI(s)'] = "https://doi.org/10.1088/0004-637X/719/2/1672"
#hennawi2010.loc[hennawi2010.Name1 == 'SDSSJ1116+4118A', 'DOI(s)'] = "https://doi.org/10.1086/498235 ; https://doi.org/10.1111/j.1365-2966.2007.11847.x"

## Taking into account the overlap between Hennawi+2006 and Miller+2004
#hennawi2010.loc[hennawi2010.Name1 == '2QZJ1435+0008A', 'Paper(s)'] = "Miller+2004 ; Hennawi+2006"
#hennawi2010.loc[hennawi2010.Name1 == '2QZJ1435+0008A', 'Bibcode(s)'] = "2004MNRAS.348..395M ; 2006AJ....131....1H"
#hennawi2010.loc[hennawi2010.Name1 == '2QZJ1435+0008A', 'DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2004.07303.x ; https://doi.org/10.1086/498235"

## Taking into account the overlap between Pindor+2006 and Hennawi+2006: Pindor+2006 identified the dual quasar \
## SDSSJ1600+0000A. (Pindor is also an author on Hennawi's paper)
#hennawi2010.loc[hennawi2010.Name1=='SDSSJ1600+0000A', 'Paper(s)'] = "Pindor+2006 ; Hennawi+2006"
#hennawi2010.loc[hennawi2010.Name1=='SDSSJ1600+0000A', 'BibCode(s)'] = "2006AJ....131...41P ; 2006AJ....131....1H"
#hennawi2010.loc[hennawi2010.Name1=='SDSSJ1600+0000A', 'DOI(s)'] = "https://doi.org/10.1086/497965 ; https://doi.org/10.1086/498235"

#hennawi2010['PairID'] = hennawi2010.index

# Now removing all systems which do not meet our cut in velocity difference of ≤2000 km s**-1
#hennawi2010 = (hennawi2010[np.abs(hennawi2010['dV_rwp'])<=2000]).reset_index(drop=True)
# (This leaves 284 pairs)

## Here we're dropping the obvious duplicates where the A and B components are not mixed up between tables
#hennawi2010 = hennawi2010.drop_duplicates(subset=['Name1','Name2','z1','z2','theta','RA1','Dec1','RA2','Dec2'])
#hennawi2010 = hennawi2010.drop_duplicates(subset=['Name1','Name2']) #removing duplicates that have correct components only
#
## Here we need to do something clever, because Hennawi+2006 mixed up the A and B designations between different \
## tables I'm going to make two new columns: sum of z's and sum of gmag's
#hennawi2010['summedz'] = hennawi2010['z1']+hennawi2010['z2']
#hennawi2010['summedgmag'] = hennawi2010['gmag1']+hennawi2010['gmag2']
#
## Now this allows me to circumvent the issue of mixed up redshift and gmag columns. Nothing I can do about the \
## mixed up names, but NONE of the rows should have duplicate summed redshifts AND gmags AND identical separations \
## unless the rows are in fact the same pair.
#hennawi2010 = hennawi2010.drop_duplicates(subset=['summedz','summedgmag','theta'])
#
## Now dropping any remaining irrelevant columns:
#hennawi2010.drop(labels=['Sample','dV','chi2','PairID','summedz','summedgmag'], axis=1, inplace=True)

coordconvert = SkyCoord(ra = hennawi2010['RA1'], dec = hennawi2010['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2010['RA1_deg'] = coordconvert.ra.degree
hennawi2010['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = hennawi2010['RA2'], dec = hennawi2010['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

hennawi2010['RA2_deg'] = coordconvert.ra.degree
hennawi2010['Dec2_deg'] = coordconvert.dec.degree

#hennawi2010.reset_index(drop=True, inplace=True)

hennawi2010

,[HMS2010]1,gmag1,chi2s1,chi2p1,z1,ez1,DelV1,DelT1,RT1,[HMS2010]2,gmag2,chi2s2,chi2p2,z2,ez2,DelV2,DelT2,RT2,RA1,Dec1,RA2,Dec2,Equinox,Coordinate_waveband,Coordinate_Source,System Type,z1_type,z2_type,Brightness_band1,Brightness_band2,Brightness_type1,Brightness_type2,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),RA1_deg,Dec1_deg,RA2_deg,Dec2_deg
0,SDSS J0004-0844A,20.75,1.2,0.9,2.998,1450,650,4.4,35,SDSS J0004-0844B,21.02,4.6,0.3,2.989,1450,<NA>,NaN,<NA>,0:4:50.674,-8:44:51.97,0:4:50.92,-8:44:51.97,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,1.211142,-8.747769,1.212167,-8.747769
1,SDSS J0829+2927A,20.93,63.3,22.5,3.054,1450,270,40.3,322,SDSS J0829+2927B,20.07,36.8,1.2,3.050,710,<NA>,NaN,<NA>,8:29:7.722,+29:27:18.1,8:29:9.018,+29:27:18.1,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,127.282175,29.455028,127.287575,29.455028
2,SDSS J0956+2643A,19.43,143.2,2.6,3.087,520,150,16.5,131,SDSS J0956+2643B,20.78,14.1,0.6,3.085,520,<NA>,NaN,<NA>,9:56:27.149,+26:43:21.62,9:56:25.934,+26:43:21.62,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,149.113121,26.722672,149.108058,26.722672
3,SDSS J0959+1032A,20.92,215.2,3.5,4.024,1450,250,44.1,320,SDSS J0959+1033B,21.47,24.5,1.7,4.020,1500,<NA>,NaN,<NA>,9:59:3.843,+10:32:25.02,9:59:5.139,+10:33:25.02,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,149.766012,10.540283,149.771412,10.556950
4,SDSS J1016+4040A,19.42,109.8,8.1,2.996,520,1030,68.2,548,SDSS J1016+4040B,20.36,49.7,1.8,2.983,1450,<NA>,NaN,<NA>,10:16:1.509,+40:40:5.8,10:16:5.842,+40:40:5.8,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,154.006288,40.668278,154.024342,40.668278
5,SDSS J1021+1112A,21.99,26.7,0.3,3.805,520,1470,7.6,56,SDSS J1021+1112B,22.36,58.2,5.4,3.829,630,<NA>,NaN,<NA>,10:21:16.982,+11:12:27.83,10:21:16.468,+11:12:27.83,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,155.320758,11.207731,155.318617,11.207731
6,SDSS J1053+5001A,21.05,18.2,0.4,3.078,630,490,2.1,17,SDSS J1053+5001B,21.10,19.4,1.5,3.085,520,<NA>,NaN,<NA>,10:53:20.15,+50:1:47.84,10:53:20.041,+50:1:47.84,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,163.333958,50.029956,163.333504,50.029956
7,SDSS J1054+0215A,20.22,244.9,6.2,3.984,630,1120,88.3,644,SDSS J1054+0216B,20.58,12.9,5.3,3.966,790,<NA>,NaN,<NA>,10:54:34.174,+2:15:16.37,10:54:28.515,+2:16:16.37,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,163.642392,2.254547,163.618812,2.271214
8,SDSS J1116+4118A,18.49,87.0,6.0,2.982,520,1840,13.8,111,SDSS J1116+4118B,19.41,87.9,1.7,3.006,520,<NA>,NaN,<NA>,11:16:11.74,+41:18:14.44,11:16:10.689,+41:18:14.44,J2000,Optical,SDSS,Binary Quasar,spec,spec,SDSS g PSF ext-corr,SDSS g PSF ext-corr,asinh,asinh,,Optical Spectroscopy,Hennawi+2010,2010ApJ...719.1672H,https://doi.org/10.1088/0004-637X/719/2/1672,169.048917,41.304011,169.044537,41.304011
9,SDSS J1118+0202A,21.13,75.0,0.8,3.939,790,0

In [14]:
#from astropy.cosmology import FlatLambdaCDM 
#from astropy.cosmology import LambdaCDM #Creating our choice of cosmology here...
#cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

#value = (30)*(cosmo.arcsec_per_kpc_proper(1.5))**(-1)

In [15]:
#(30)*(cosmo.arcsec_per_kpc_proper(1.5))**(-1)

In [16]:
#print(value.value)

In [12]:
# Here we're loading in the catalog(s) from Myers+2007:

myers2007 = ((Table.read('Myers_2007/table1.dat', readme = 'Myers_2007/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['SDSSa','SDSSb']) 
# I've edited the ReadMe to remove the duplicate columns '---' which just contain the string 'SDSS' for each row
# They load in as SDSSa and SDSSb now, and they are immediately dropped from the dataframe (above). 

# (a) DR4CatalogArchive Server (e.g., likely Schneider et al. 2005). (b) Hennawi et al. (2006). \
# (c) Sramek & Weedman (1978) and Djorgovski & Spinrad (1984). (d ) Probable binary quasar-starburst \
# (Brotherton et al. 1999).

# Now combining the various coordinate columns to give us a RA and Dec column for each quasar in each pair
myers2007['RA1'] = myers2007['RAh'].astype(str) + ':' + myers2007['RAm'].astype(str) + ':' + myers2007['RAs'].astype(str)
myers2007['Dec1'] = myers2007['DE-'].astype(str) + myers2007['DEh'].astype(str) + ':' + myers2007['DEm'].astype(str) + ':' + myers2007['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = myers2007['RA1'], dec = myers2007['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

myers2007['RA1_deg'] = coordconvert.ra.degree
myers2007['Dec1_deg'] = coordconvert.dec.degree

# Now for the second set of coordinates
myers2007['RA2'] = myers2007['RA2h'].astype(str) + ':' + myers2007['RA2m'].astype(str) + ':' + myers2007['RA2s'].astype(str)
myers2007['Dec2'] = myers2007['DE2-'].astype(str) + myers2007['DE2h'].astype(str) + ':' + myers2007['DE2m'].astype(str) + ':' + myers2007['DE2s'].astype(str)

coordconvert = SkyCoord(ra = myers2007['RA2'], dec = myers2007['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

myers2007['RA2_deg'] = coordconvert.ra.degree
myers2007['Dec2_deg'] = coordconvert.dec.degree

# Adding in columns for information about the coordinates
myers2007['Equinox'] = "J2000"
myers2007['Coordinate_waveband'] = "Optical"
myers2007['Coordinate_Source'] = "SDSS"

myers2007['System Type'] = 'Binary Quasar'
myers2007['Selection Method'] = 'Optical Photometry'
myers2007['Confirmation Method'] = -100

myers2007['Brightness_band1'] = 'SDSS g'
myers2007['Brightness_type1'] = 'asinh mag'

myers2007['Brightness_band2'] = 'SDSS g'
myers2007['Brightness_type2'] = 'asinh mag'

# Now dropping the original 14 columns for separate RA and Dec components
myers2007.drop(labels=['RAh','RAm','RAs','DE-','DEh','DEm','DEs','RA2h','RA2m','RA2s','DE2-','DE2h','DE2m','DE2s'], axis=1, inplace=True)

# There is some overlap between Myers+2007 and other studies, like Hennawi+2006. 
# We will prioritize spectroscopic results over photometric results for our catalog. We see that some of the \
# overlap targets between Myers+ and other studies are noted as being quasar star pairs. Since these pairs \
# overlap with the spectroscopic study of Hennawi+ and others, we will remove these and assume these are genuinely \
# quasar star pairs, and therefore are not relevant for our study. 

# Removing the quasar-star pairs now...
myers2007 = (myers2007[(myers2007['n_zsp2']!="star b") & (myers2007['f_zsp1']!="star a")]).reset_index(drop=True)

# Brotheron+1999 list the redshift of both systems as being z=0.586, but Myers+07 notes only that System A has this
# spec-z. Here I am replacing the NaN values for zsp2 and N-zsp2...
myers2007.loc[myers2007.f_zsp1 == 'a,d', 'zsp2'] = 0.586
myers2007.loc[myers2007.f_zsp1 == 'a,d', 'n_zsp2'] = 'd'

# Making now a copy of this to compare to other studies and then removing all listings from other studies from the \
# main Myers+07 dataframe
myers2007_archive = myers2007.copy()

# Now we're going to split this dataframe in two: one with photometric pairs (which might have one spec-z) and\
# one where we have only spec-z pairs from other literature sources

myers2007_archive['zsp1'] = myers2007_archive['zsp1'].fillna(0)
myers2007_archive['zsp1b'] = myers2007_archive['zsp1b'].fillna(0)
myers2007_archive['zsp2'] = myers2007_archive['zsp2'].fillna(0)
myers2007_archive['zsp2b'] = myers2007_archive['zsp2b'].fillna(0)

# This one above has the spec-z pairs
myers2007_spec = myers2007_archive[((myers2007_archive['zsp1']>0) | (myers2007_archive['zsp1b']>0)) & ((myers2007_archive['zsp2']>0) | (myers2007_archive['zsp2b']>0))]
# This one below has the photometric pairs (where one may or may not have a spec-z)
myers2007_phot = (myers2007_archive.loc[myers2007_archive.index.difference(myers2007_spec.index)]).reset_index(drop=True)

myers2007_spec = (myers2007_spec).reset_index(drop=True) # Resetting the indices now...
myers2007_phot = (myers2007_phot).reset_index(drop=True)

# At this point, we essentially won't be using the _spec dataframe from Myers2007 as we will favor the data from \
# the spectroscopic studies instead.
# I manually checked the overlap between Myers+ and Hennawi+; the angular separations appear to agree between \
# the two paper, so I'll just stick with Hennawi's calculations

# Also note that the the columns zsp1b and zsp2b do not contain any additional redshift information, so we'll \
# drop the columns zsp1b, r_zsp1b, zsp2b, and r_zsp2b. We'll also drop the umag1 and umag2 columns
myers2007_phot.drop(labels=['zsp1b', 'r_zsp1b', 'zsp2b', 'r_zsp2b', 'umag1', 'umag2'], axis=1, inplace=True)

# Since we're not interested in the spectroscopic pairs from this table (they're included elsewhere, and no \
# new information is provided by Myers+07), I'm going to overwrite the dataframe myers2007 such that \
# myers2007 = myers2007_phot
myers2007 = myers2007_phot

# Now creating two columns: one for adopted redshift and one for adopted redshift type
# For sources with a spec-z, we'll adopt the spec-z. Otherwise we'll adopt the **MIDPOINT** phot-z of Myers+.

myers2007['z1'] = myers2007['zph1m'].where((myers2007['zsp1']==0), other=myers2007['zsp1'])
myers2007['z1_type'] = 'phot'
myers2007.loc[myers2007.zsp1>0, 'z1_type'] = 'spec'

myers2007['z2'] = myers2007['zph2m'].where((myers2007['zsp2']==0), other=myers2007['zsp2'])
myers2007['z2_type'] = 'phot'
myers2007.loc[myers2007.zsp2>0, 'z2_type'] = 'spec'

# Adding in information for the paper and catalog...
myers2007['Paper(s)'] = "Myers+2007"
myers2007['BibCode(s)'] = "2007ApJ...658...99M"
myers2007['DOI(s)'] = "https://doi.org/10.1086/511520"

# Keeping this here for now...
myers2007['PairID'] = myers2007.index



In [13]:
# 111 total pairs in Myers+2007
# --> 20 are spectroscopic pairs that overlap with other studies; removed, leaving 91 pairs.
# --> 3 instances of other studies finding that one of the 'quasars' was a star; removed, leaving 88 pairs

In [14]:
# Here we're loading in the catalog(s) from Myers+2008:

myers2008_t1 = ((Table.read('Myers_2008/table1.dat', readme = 'Myers_2008/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# Table 1 contains DR4 KDE candidate binaries for which only one member has been observed to date

myers2008_t2 = ((Table.read('Myers_2008/table2.dat', readme = 'Myers_2008/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# Table 2 contains confirmed binary quasars from DR4 KDE candidate sample

myers2008_t3 = (Table.read('Myers_2008/table3.dat', readme = 'Myers_2008/ReadMe', format='ascii.cds')).to_pandas()
# Table 3 contains the 10 projected pairs from the DR4 KDE candidate sample

myers2008_t4 = ((Table.read('Myers_2008/table4.dat', readme = 'Myers_2008/ReadMe', format='ascii.cds')).to_pandas()).drop(columns=['---'])
# Table 4 contains ambiguous pairs in the DR4 KDE binary quasar candidate sample

myers2008_t5 = (Table.read('Myers_2008/table5.dat', readme = 'Myers_2008/ReadMe', format='ascii.cds')).to_pandas()
# Table 5 contains previously identified DR4 KDE binary quasar candidates with 3''<theta<6'' separations.

# Now breaking this table into different tables to make removing sources easier...
myers2008_t5_pp = myers2008_t5[myers2008_t5['oc']=='PP'] # 12 projected pairs / non pairs
myers2008_t5_BQ = myers2008_t5[myers2008_t5['oc']=='BQ']
myers2008_t5_CL = myers2008_t5[myers2008_t5['oc']=='CL'] # 2 lenses

# Immediately we need to remove the quasars from Table 3 and the PP and CL quasars from Table 5 from the \
# Myers2007 catalog, which we'll do now...

# Note, the BQ's from Table 5 were already removed when we removed spectroscopic pairs that overlapped with \
# other catalogs/other papers.

In [15]:
# Removing the 10 projected pairs from Table 3 first...
myers_a = (myers2007[~myers2007['SDSS1'].isin(myers2008_t3['SDSS'])]).reset_index(drop=True)
myers_b = (myers_a[~myers_a['SDSS2'].isin(myers2008_t3['SDSS'])]).reset_index(drop=True)

# Now removing 12 projected pairs from Table 5. Some of these overlap with previously removed systems...
myers_c = (myers_b[~myers_b['SDSS1'].isin(myers2008_t5_pp['SDSS'])]).reset_index(drop=True)
myers_d = (myers_c[~myers_c['SDSS1'].isin(myers2008_t5_pp['SDSS'])]).reset_index(drop=True)
# This leaves us with 76 pairs...

# Now removing the 2 confirmed lenses from Table 5
# Based upon the coordinates listed in Myers+2008 Table 5, the sources have coordinates:
# 10 04 34.800 +41 12 39.29 --> SDSS1 = J100434.79+411239.2 (Verified manually)
# 12 06 29.648 +43 32 17.57 --> SDSS1 = J120629.65+433220.6 (verified manually)
lenses = pd.DataFrame({'SDSS1': ['J100434.79+411239.2','J120629.65+433220.6']})
myers_e = (myers_d[~myers_d['SDSS1'].isin(lenses['SDSS1'])]).reset_index(drop=True)
# This leaves us with 74 pairs


In [16]:
# Now manually updating the myers table (by matching their new nomenclature to the old by eye) for the \
# CONFIRMED binaries (spec-z's) from Table 2

# For J1158+1235A,B
# J115822.77+123518.5 --> 1158 A
# J115822.98+123520.3 --> 1158 B

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'zsp1'] = myers2008_t2['z'].iloc[0]
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'zsp2'] = myers2008_t2['z'].iloc[1]

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z1'] = myers2008_t2['z'].iloc[0]
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z2'] = myers2008_t2['z'].iloc[1]

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e['Notes'] = ''
myers_e.loc[myers_e.SDSS1=='J115822.77+123518.5', 'Notes'] = 'Spec-z from Myers+2008; J115822.77+123518.5: 1158 A and 115822.98+123520.3: 1158 B.'


# For J1320+3056A,B
# J132022.54+305622.8 --> J1320+3056A (gmag=18.60)
# J132022.64+305618.2 --> J1320+3056B (gmag=19.92)

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'zsp1'] = myers2008_t2['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'zsp2'] = myers2008_t2['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z1'] = myers2008_t2['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z2'] = myers2008_t2['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J132022.54+305622.8', 'Notes'] = 'Spec-z from Myers+2008; J132022.54+305622.8: 1320 A and J132022.64+305618.2: 1320 B.'


# For J1418+2441A,B
# J141855.41+244108.9 --> J1418+2441A (gmag=19.27)
# J141855.53+244104.7 --> J1418+2441B (gmag=20.22)

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'zsp1'] = myers2008_t2['z'].iloc[4]
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'zsp2'] = myers2008_t2['z'].iloc[5]

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z1'] = myers2008_t2['z'].iloc[4]
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z2'] = myers2008_t2['z'].iloc[5]

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J141855.41+244108.9', 'Notes'] = 'Spec-z from Myers+2008; J141855.41+244108.9: 1418 A and 141855.53+244104.7: 1418 B.'


# For J1426+0719A,B
# J142604.32+071930.0 --> J1426+0719B (gmag=20.12)
# J142604.26+071925.8 --> J1426+0719A (gmag=20.82)

# Bear in mind that we've had to flip the index order, as the A/B order flipped between catalogs
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'zsp1'] = myers2008_t2['z'].iloc[7]
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'zsp2'] = myers2008_t2['z'].iloc[6]

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z1'] = myers2008_t2['z'].iloc[7]
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z2'] = myers2008_t2['z'].iloc[6]

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS2=='J142604.26+071925.8', 'Notes'] = 'Spec-z from Myers+2008; J142604.26+071925.8: 1426 A and J142604.32+071930.0: 1426 B.'


# For J1430+0714A,B
# J143002.88+071411.3 --> J1430+0714B (gmag=19.50)
# J143002.66+071415.6 --> J1430+0714A (gmag=20.27)

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'zsp1'] = myers2008_t2['z'].iloc[9]
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'zsp2'] = myers2008_t2['z'].iloc[8]

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z1'] = myers2008_t2['z'].iloc[9]
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z2'] = myers2008_t2['z'].iloc[8]

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'Paper(s)'] = myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'DOI(s)'] = myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS2=='J143002.66+071415.6', 'Notes'] = 'Spec-z from Myers+2008; J143002.66+071415.6: 1430 A and J143002.88+071411.3: 1430 B.'


# J1458+5448A,B
# J145826.72+544813.1 --> J1458+5448B (gmag=20.53)
# J145826.16+544814.8 --> J1458+5448A (gmag=20.79)

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'zsp1'] = myers2008_t2['z'].iloc[11]
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'zsp2'] = myers2008_t2['z'].iloc[10]

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z1'] = myers2008_t2['z'].iloc[11]
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z2'] = myers2008_t2['z'].iloc[10]

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J145826.72+544813.1', 'Notes'] = 'Spec-z from Myers+2008; J145826.72+544813.1: 1458 B and J145826.16+544814.8: 1458 A.'


# J1606+2900A,B
# J160603.02+290050.8 --> J1606+2900B (gmag=18.42)
# J160602.81+290048.7 --> J1606+2900A (gmag=18.50)

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'zsp1'] = myers2008_t2['z'].iloc[13]
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'zsp2'] = myers2008_t2['z'].iloc[12]

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z1'] = myers2008_t2['z'].iloc[13]
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z2'] = myers2008_t2['z'].iloc[12]

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J160603.02+290050.8', 'Notes'] = 'Spec-z from Myers+2008; J160603.02+290050.8: 1606 B and J160602.81+290048.7: 1606 A.'


# J1635+2911A,B
# J163510.30+291116.1 --> 1635+2911B (gmag=20.43)
# J163510.14+291120.6 --> 1635+2911A (gmag=18.83)

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'zsp1'] = myers2008_t2['z'].iloc[15]
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'zsp2'] = myers2008_t2['z'].iloc[14]

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z1'] = myers2008_t2['z'].iloc[15]
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z2'] = myers2008_t2['z'].iloc[14]

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J163510.30+291116.1', 'Notes'] = 'Spec-z from Myers+2008; J163510.30+291116.1: 1635 B and J163510.14+291120.6: 1635 A.'



In [17]:
# Now adding in the spec-z's from the more ambiguous pairs in Table 4

# SDSS1=J093424.32+421130.8 and SDSS2=J093424.11+421135.0

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'zsp1'] = myers2008_t4['z'].iloc[1]
#myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'zsp2'] = myers2008_t4['z'].iloc[0] #This does not have a listed redshift

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z1'] = myers2008_t4['z'].iloc[1]
#myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z2'] = myers2008_t4['z'].iloc[0] # No listed redshift

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z1_type'] = 'spec'
#myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'z2_type'] = 'spec' # No spec-z listed

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J093424.32+421130.8', 'Notes'] = 'Spec-z from Myers+2008. Follow-up spectroscopy likely needed.'


# SDSS1=J120727.09+140817.1 and SDSS2=J120727.25+140820.3

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'zsp1'] = myers2008_t4['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'zsp2'] = myers2008_t4['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z1'] = myers2008_t4['z'].iloc[2]
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z2'] = myers2008_t4['z'].iloc[3]

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J120727.09+140817.1', 'Notes'] = 'Spec-z from Myers+2008. Follow-up spectroscopy likely needed.'


# SDSS1=J123555.27+683627.0 --> J1235+6836B (gmag=)19.70
# SDSS2=J123554.78+683624.7 --> J1235+6836A (gmag=19.04)

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'zsp1'] = myers2008_t4['z'].iloc[5]
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'zsp2'] = myers2008_t4['z'].iloc[4]

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z1'] = myers2008_t4['z'].iloc[5]
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z2'] = myers2008_t4['z'].iloc[4]

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J123555.27+683627.0', 'Notes'] = 'Spec-z from Myers+2008; J123555.27+683627.0: J1235 B and J123554.78+683624.7: J1235 A. Follow-up spectroscopy likely needed.'


# SDSS1=J150747.23+290333.2 --> J1507+2903B (gmag=19.97)
# SDSS2=J150746.90+290334.1 --> J1507+2903A (gmag=20.44)

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'zsp1'] = myers2008_t4['z'].iloc[7]
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'zsp2'] = myers2008_t4['z'].iloc[6]

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z1'] = myers2008_t4['z'].iloc[7]
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z2'] = myers2008_t4['z'].iloc[6]

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z1_type'] = 'spec'
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'z2_type'] = 'spec'

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'Paper(s)'] = myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'Paper(s)'] + ' ; Myers+2008'
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'BibCode(s)'] = myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'DOI(s)'] = myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'

myers_e.loc[myers_e.SDSS1=='J150747.23+290333.2', 'Notes'] = 'Spec-z from Myers+2008; J150747.23+290333.2: J1507 B and J150746.90+290334.1: J1507 A. Follow-up spectroscopy likely needed.'



In [18]:
# Here, we will break the myers master catalog into two: one for spec-z pairs, and the other for phot-z pairs or \
# phot-z/spec-z pairs

# This way, the spec pairs will be included in a uniform fashion with those in the master table, while a secondary \
# table for phot-z pairs will be included with the explicit mention that these require optical spectroscopic \
# follow-up

myers_master_spec = (myers_e[(myers_e['z1_type']=='spec') & (myers_e['z2_type']=='spec')]).reset_index(drop=True)
myers_master_phot = (myers_e[~myers_e['SDSS1'].isin(myers_master_spec['SDSS1'])]).reset_index(drop=True)

# And now we can remove the irrlevant photometric redshift columns and redundant spec-z columns from the \
# myers_master_spec table

myers_master_spec.drop(labels=['zph1l','zph1m','zph1u','zsp1','f_zsp1','zph2l','zph2m','zph2u','zsp2','n_zsp2',\
                              'PairID','f_Delta'], axis=1, inplace=True)
myers_master_spec['Confirmation Method'] = 'Optical Spectroscopy'

myers_master_phot.drop(labels=['PairID'], axis=1, inplace=True)
myers_master_phot['Notes'] = 'Spectroscopic follow-up is required.'
myers_master_phot['System Type'] = 'Binary Quasar Candidate'

In [19]:
# And we need to update the phot table with any remaining spec-z's that Myers found (see Table 1)
# There are 8 systems in which they have spec-z measurements for 1 out of 2 components in each pair

# SDSS1= J093015.01+420033.6  (gmag=20.03)
# SDSS2= J093014.81+420038.7 --> Has spec-z in Table 1 (gmag=19.71)

myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'zsp2'] = myers2008_t1['z'].iloc[0]
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'z2'] = myers2008_t1['z'].iloc[0]
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'z2_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS2=='J093014.81+420038.7', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J093521.02+641219.8 --> Has spec-z in Table 1 (gmag=20.99)
# SDSS2=J093521.80+641221.9   (gmag=20.96)

myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'zsp1'] = myers2008_t1['z'].iloc[2]
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'z1'] = myers2008_t1['z'].iloc[2]
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'z1_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS1=='J093521.02+641219.8', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J095840.74+332216.3 --> Has spec-z in Table 1 (gmag=19.18)
# SDSS2=J095840.94+332211.5   (gmag=20.64)

myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'zsp1'] = myers2008_t1['z'].iloc[4]
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'z1'] = myers2008_t1['z'].iloc[4]
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'z1_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS1=='J095840.74+332216.3', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J103939.31+100253.0 --> Has spec-z in Table 1 (gmag=18.42)
# SDSS2=J103939.53+100254.3  (gmag=19.60)

myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'zsp1'] = myers2008_t1['z'].iloc[6]
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'z1'] = myers2008_t1['z'].iloc[6]
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'z1_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS1=='J103939.31+100253.0', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'


# SDSS1=J162847.75+413045.4  (gmag=19.81)
# SDSS2=J162848.06+413043.1 --> Has a spec-z in Table 1 (gmag=20.40)

myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'zsp2'] = myers2008_t1['z'].iloc[9]
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'z2'] = myers2008_t1['z'].iloc[9]
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'z2_type'] = 'spec'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'Paper(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'Paper(s)'] + ' ; Myers+2008'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'BibCode(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'BibCode(s)'] + ' ; 2008ApJ...678..635M'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'DOI(s)'] = myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'DOI(s)'] + ' ; https://doi.org/10.1086/533491'
myers_master_phot.loc[myers_master_phot.SDSS2=='J162848.06+413043.1', 'Notes'] = 'Spec-z from Myers+2008. Spectroscopic follow-up is required.'



In [20]:
# Now constructing the remaining columns for delta_z and dV for the spec pairs...
myers_master_spec['delta_z'] = myers_master_spec['z1']-myers_master_spec['z2']
myers_master_spec['dV'] = (2.99e+5)*((1+myers_master_spec['z1'])**2 - (1+myers_master_spec['z2'])**2)/((1+myers_master_spec['z1'])**2+(1+myers_master_spec['z2'])**2)
myers_master_spec['Sep(kpc)'] = myers_master_spec['Delta']*((cosmo.arcsec_per_kpc_proper(myers_master_spec['z1']))**(-1))

# Now renaming / rearranging columns for these two tables (to make concatenation easier later on...)

# First the spec table
myers_master_spec = myers_master_spec[['System Type','SDSS1','SDSS2','Selection Method','Confirmation Method',\
                                       'z1','z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2',\
                                       'Dec2','RA2_deg','Dec2_deg','Equinox','Coordinate_waveband',\
                                       'Coordinate_Source','gmag1','Brightness_band1','Brightness_type1','gmag2',\
                                       'Brightness_band2','Brightness_type2','Delta','Sep(kpc)','delta_z','dV',\
                                       'Paper(s)','BibCode(s)','DOI(s)','Notes']]

myers_master_spec.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method',\
                           'z1','z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2',\
                           'Dec2','RA2_deg','Dec2_deg','Equinox','Coordinate_waveband',\
                           'Coordinate_Source','Brightness1','Brightness_band1','Brightness_type1','Brightness2',\
                           'Brightness_band2','Brightness_type2','Sep','Sep(kpc)','delta_z','dV',\
                           'Paper(s)','BibCode(s)','DOI(s)','Notes']

# Now the phot table
myers_master_phot = myers_master_phot[['System Type','SDSS1','SDSS2','Selection Method','Confirmation Method',\
                                       'zph1l','zph1m','zph1u','zsp1','f_zsp1','z1','z1_type','zph2l','zph2m',\
                                       'zph2u','zsp2','n_zsp2','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg',\
                                       'RA2','Dec2','RA2_deg', 'Dec2_deg','Equinox','Coordinate_waveband',\
                                       'Coordinate_Source','gmag1','Brightness_band1','Brightness_type1','gmag2',\
                                       'Brightness_band2','Brightness_type2','Delta','f_Delta','Paper(s)',\
                                       'BibCode(s)','DOI(s)','Notes']]

myers_master_phot.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method',\
                           'zph1l','zph1m','zph1u','zsp1','f_zsp1','z1','z1_type','zph2l','zph2m',\
                           'zph2u','zsp2','n_zsp2','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg',\
                           'RA2','Dec2','RA2_deg','Dec2_deg','Equinox','Coordinate_waveband',\
                           'Coordinate_Source','Brightness1','Brightness_band1','Brightness_type1','Brightness2',\
                           'Brightness_band2','Brightness_type2','Sep','f_Sep','Paper(s)',\
                           'BibCode(s)','DOI(s)','Notes']
        

In [26]:
#myers_master_phot.columns

In [27]:
#myers_master_spec

In [330]:
myers_master_phot

,System Type,Name1,Name2,Selection Method,Confirmation Method,zph1l,zph1m,zph1u,zsp1,f_zsp1,z1,z1_type,zph2l,zph2m,zph2u,zsp2,n_zsp2,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,f_Sep,Paper(s),BibCode(s),DOI(s),Notes
0,Binary Quasar Candidate,J024037.34-070626.3,J024037.11-070623.9,Optical Photometry,-100,1.45,1.875,1.95,0.000,NaN,1.875,phot,1.45,1.575,1.80,0.000,NaN,1.575,phot,2:40:37.35,-7:6:26.3,40.155625,-7.107306,2:40:37.11,-7:6:23.9,40.154625,-7.106639,J2000,Optical,SDSS,20.51,SDSS g,asinh mag,20.98,SDSS g,asinh mag,4.30,NaN,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
1,Binary Quasar Candidate,J034134.90-063150.3,J034134.78-063145.7,Optical Photometry,-100,1.40,1.925,2.10,0.000,NaN,1.925,phot,1.05,1.425,1.55,0.000,NaN,1.425,phot,3:41:34.91,-6:31:50.4,55.395458,-6.530667,3:41:34.79,-6:31:45.7,55.394958,-6.529361,J2000,Optical,SDSS,19.67,SDSS g,asinh mag,21.18,SDSS g,asinh mag,5.01,NaN,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
2,Binary Quasar Candidate,J071803.50+402102.6,J071803.09+402059.1,Optical Photometry,-100,1.65,1.925,2.10,0.000,NaN,1.925,phot,1.60,1.775,2.00,0.000,NaN,1.775,phot,7:18:3.51,+40:21:2.7,109.514625,40.350750,7:18:3.09,+40:20:59.2,109.512875,40.349778,J2000,Optical,SDSS,20.39,SDSS g,asinh mag,21.07,SDSS g,asinh mag,5.93,*,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
3,Binary Quasar Candidate,J073009.55+354151.9,J073009.66+354149.2,Optical Photometry,-100,1.65,1.825,2.00,0.000,NaN,1.825,phot,0.70,0.875,1.15,0.000,NaN,0.875,phot,7:30:9.56,+35:41:52.0,112.539833,35.697778,7:30:9.66,+35:41:49.2,112.540250,35.697000,J2000,Optical,SDSS,20.70,SDSS g,asinh mag,20.99,SDSS g,asinh mag,3.06,*,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
4,Binary Quasar Candidate,J075339.86+173410.6,J075340.08+173408.0,Optical Photometry,-100,0.85,0.975,1.40,0.000,NaN,0.975,phot,1.05,1.275,1.40,0.000,NaN,1.275,phot,7:53:39.87,+17:34:10.7,118.416125,17.569639,7:53:40.08,+17:34:8.0,118.417000,17.568889,J2000,Optical,SDSS,19.61,SDSS g,asinh mag,20.48,SDSS g,asinh mag,4.07,NaN,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
5,Binary Quasar Candidate,J081313.10+541647.0,J081312.63+541649.9,Optical Photometry,-100,0.65,0.775,0.90,0.000,NaN,0.775,phot,0.65,0.725,0.90,0.000,NaN,0.725,phot,8:13:13.11,+54:16:47.0,123.304625,54.279722,8:13:12.64,+54:16:49.9,123.302667,54.280528,J2000,Optical,SDSS,17.40,SDSS g,asinh mag,20.25,SDSS g,asinh mag,5.04,NaN,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
6,Binary Quasar Candidate,J081803.82+191703.4,J081804.05+191703.8,Optical Photometry,-100,1.50,1.725,2.00,0.000,NaN,1.725,phot,1.50,1.775,2.05,0.000,NaN,1.775,phot,8:18:3.83,+19:17:3.5,124.515958,19.284306,8:18:4.05,+19:17:3.8,124.516875,19.284389,J2000,Optical,SDSS,20.14,SDSS g,asinh mag,21.17,SDSS g,asinh mag,3.20,NaN,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
7,Binary Quasar Candidate,J084624.33+270958.3,J084624.50+271002.3,Optical Photometry,-100,1.90,2.175,2.25,0.000,NaN,2.175,phot,2.05,2.175,2.35,0.000,NaN,2.175,phot,8:46:24.34,+27:9:58.4,131.601417,27.166222,8:46:24.51,+27:10:2.4,131.602125,27.167333,J2000,Optical,SDSS,20.66,SDSS g,asinh mag,20.62,SDSS g,asinh mag,4.64,NaN,Myers+2007,2007ApJ...658...99M,https://doi.org/10.1086/511520,Spectroscopic follow-up is required.
8,Binary Quasar Candidate,J085011.87+093122.0,J085011.82+093119.6,Optical Photometry,-100,0.40,0.675,0.90,0.000,NaN,0.675,phot,0.65,0.775,0.95,0.000,NaN,0.775,phot,8:50:11.87,+9:31:22.1,132.549458,9.522806,8:50:11.82,+9:31:19.7,132.549250,9.522139,J

In [ ]:
# Adding in citations for Vignali+2018 here

vigobjs = ['SDSS J1622+0702A','SDSS J1307+0422A']

# Adding the DOI, author, and bibcode info 
for index, row in thewhills.iterrows():
    if row['Name1'] in vigobjs:
        the_whills.at[index, 'Paper(s)'] += ' ; Vignali+2018'
        the_whills.at[index, 'BibCode(s)'] += ' ; 2018MNRAS.477..780V' 
        the_whills.at[index, 'DOI(s)'] += ' ; https://doi.org/10.1093/mnras/sty682'
        the_whills.at[index, 'Notes'] += ' The BAL QSO in this system is heavily obscured in the X-rays (>10^23)'




In [21]:
# Now loading in the catalog(s) from Kirkman+2008
kirkman2008 = (Table.read('Kirkman_2008/table1.dat', readme = 'Kirkman_2008/ReadMe', format='ascii.cds')).to_pandas()

# Creating RA and Dec columns for the foreground and background quasars
kirkman2008['RA1'] = kirkman2008['RAh'].astype(str) + ':' + kirkman2008['RAm'].astype(str) + ':' + kirkman2008['RAs'].astype(str)
kirkman2008['Dec1'] = kirkman2008['DE-'].astype(str) + kirkman2008['DEd'].astype(str) + ':' + kirkman2008['DEm'].astype(str) + ':' + kirkman2008['DEs'].astype(str)

kirkman2008['RA2'] = kirkman2008['RA2h'].astype(str) + ':' + kirkman2008['RA2m'].astype(str) + ':' + kirkman2008['RA2s'].astype(str)
kirkman2008['Dec2'] = kirkman2008['DE2-'].astype(str) + kirkman2008['DE2d'].astype(str) + ':' + kirkman2008['DE2m'].astype(str) + ':' + kirkman2008['DE2s'].astype(str)

# Now dropping the original 14 columns for separate RA and Dec components
kirkman2008.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs','RA2h','RA2m','RA2s','DE2-','DE2d','DE2m','DE2s'], axis=1, inplace=True)

# Dropping irrelevant columns, such as S/N columns 'S', 'S2', as well as the impact parameter 'b', the Lyman-alpha
# luminosity limit 'LLya', and the expected UV flux ratio 'OmeMax'
kirkman2008.drop(labels=['S', 'S2','b','LLya','OmeMax'], axis=1, inplace=True)

# Adding in columns for information about the coordinates
kirkman2008['Equinox'] = "J2000"
kirkman2008['Coordinate_waveband'] = "NA"
kirkman2008['Coordinate_Source'] = "NA"

kirkman2008['Brightness_type1'] = '-100'
kirkman2008['Brightness_type2'] = '-100'

# Calculating the redshift difference and velocity difference between the quasar pairs
kirkman2008['delta_z'] = kirkman2008['zem'] - kirkman2008['zem2']
kirkman2008['dV'] = (2.99e+5)*((1+kirkman2008['zem'])**2 - (1+kirkman2008['zem2'])**2)/((1+kirkman2008['zem'])**2+(1+kirkman2008['zem2'])**2)
kirkman2008['Sep(kpc)'] = kirkman2008['Sep']*((cosmo.arcsec_per_kpc_proper(kirkman2008['zem']))**(-1))

# Now adding in columns for the author+year, bibcode, and doi link
kirkman2008['Paper(s)'] = "Kirkman+2008"
kirkman2008['bibcode(s)'] = "2008MNRAS.391.1457K"
kirkman2008['DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2008.13994.x"

# Removing quasar pairs with velocity differences |v|>2000 km/s 
kirkman2008_master = (kirkman2008[np.abs(kirkman2008['dV'])<=2000]).reset_index(drop=True)

# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = kirkman2008_master['RA1'], dec = kirkman2008_master['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

kirkman2008_master['RA1_deg'] = coordconvert.ra.degree
kirkman2008_master['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = kirkman2008_master['RA2'], dec = kirkman2008_master['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

kirkman2008_master['RA2_deg'] = coordconvert.ra.degree
kirkman2008_master['Dec2_deg'] = coordconvert.dec.degree


In [22]:
#kirkman2008_master

In [23]:
# Now loading in the catalog(s) from Tytler+2009
tytler2009 = (Table.read('Tytler_2009/table1.dat', readme = 'Tytler_2009/ReadMe', format='ascii.cds')).to_pandas()

# Table 4 looks irrelevant for our purposes, so we're not loading that in.

# Creating an RA and Dec column for each source
tytler2009['RA'] = tytler2009['RAh'].astype(str) + ':' + tytler2009['RAm'].astype(str) + ':' + tytler2009['RAs'].astype(str)
tytler2009['Dec'] = tytler2009['DE-'].astype(str) + tytler2009['DEd'].astype(str) + ':' + tytler2009['DEm'].astype(str) + ':' + tytler2009['DEs'].astype(str)

# Dropping the original 7 columns for RA and Dec
tytler2009.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs'], axis=1, inplace=True)

# Now separating pairs from triples, and taking each list - where the pairs are listed across two rows each - and \
# realigning the information so that each pair inhabits one row. This will make calculations easier.

tytler2009['PairID'] = tytler2009['Pair']

# Grouping the objects by Pair ID and breaking the table into two adjust the formatting...

tytler2009_pairs = (tytler2009.groupby('Pair').filter(lambda x: len(x['PairID'])<=2)).reset_index(drop=True)
tytler2009_pairs.drop(labels=['PairID'], axis=1, inplace=True)

tytler2009_3s = (tytler2009.groupby('Pair').filter(lambda x: len(x['PairID'])>2)).reset_index(drop=True)
tytler2009_3s.drop(labels=['PairID'], axis=1, inplace=True)

# Now slicing the pairs table and then joining them
tytler_pairs1 = tytler2009_pairs[0::2] # Slicing now only the A components
tytler_pairs2 = tytler2009_pairs[1::2] # Slicing now only the B components

tytler2009_pairs = tytler_pairs1.join(tytler_pairs2.set_index('Pair'), on='Pair', how='left', lsuffix = '1', rsuffix='2')

# Now doing the same for the triples...
tytler2009_3s1 = tytler2009_3s[0::3] # Slicing now only the A components
tytler2009_3s2 = tytler2009_3s[1::3] # Slicing now only the B components
tytler2009_3s3 = tytler2009_3s[2::3] # Slicing now only the C components

# Renaming the columns for the triplets so that they are easier to work with
tytler2009_3s1.columns=['zem1','---1','Pair','m_Pair1','Sep12','RA1','Dec1']
tytler2009_3s2.columns=['zem2','---2','Pair','m_Pair2','Sep23','RA2','Dec2']
tytler2009_3s3.columns=['zem3','---3','Pair','m_Pair3','Sep13','RA3','Dec3']

# Combining the triplets together
tytler2009_3s = tytler2009_3s1.join(tytler2009_3s2.set_index('Pair'), on='Pair', how='left', lsuffix = '1', rsuffix='2')
tytler2009_3 = tytler2009_3s.join(tytler2009_3s3.set_index('Pair'), on='Pair', how='left', rsuffix='3')

# Adding in columns for information about the coordinates
tytler2009_pairs['Equinox'] = "J2000"
tytler2009_pairs['Coordinate_waveband'] = "NA"
tytler2009_pairs['Coordinate_Source'] = "NA"

tytler2009_3['Equinox'] = "J2000"
tytler2009_3['Coordinate_waveband'] = "NA"
tytler2009_3['Coordinate_Source'] = "NA"

# Calculating the redshift difference and velocity difference between the quasar pairs
tytler2009_pairs['delta_z'] = tytler2009_pairs['zem1'] - tytler2009_pairs['zem2']
tytler2009_pairs['dV'] = (2.99e+5)*((1+tytler2009_pairs['zem1'])**2 - (1+tytler2009_pairs['zem2'])**2)/((1+tytler2009_pairs['zem1'])**2+(1+tytler2009_pairs['zem2'])**2)
tytler2009_pairs['Sep(kpc)'] = tytler2009_pairs['Sep1']*((cosmo.arcsec_per_kpc_proper(tytler2009_pairs['zem1']))**(-1))

tytler2009_3['delta_z12'] = tytler2009_3['zem1'] - tytler2009_3['zem2']
tytler2009_3['dV12'] = (2.99e+5)*((1+tytler2009_3['zem1'])**2 - (1+tytler2009_3['zem2'])**2)/((1+tytler2009_3['zem1'])**2+(1+tytler2009_3['zem2'])**2)
tytler2009_3['Sep(kpc)12'] = tytler2009_3['Sep12']*((cosmo.arcsec_per_kpc_proper(tytler2009_3['zem1']))**(-1))

tytler2009_3['delta_z23'] = tytler2009_3['zem2'] - tytler2009_3['zem3']
tytler2009_3['dV23'] = (2.99e+5)*((1+tytler2009_3['zem2'])**2 - (1+tytler2009_3['zem3'])**2)/((1+tytler2009_3['zem2'])**2+(1+tytler2009_3['zem3'])**2)
tytler2009_3['Sep(kpc)23'] = tytler2009_3['Sep23']*((cosmo.arcsec_per_kpc_proper(tytler2009_3['zem2']))**(-1))

tytler2009_3['delta_z13'] = tytler2009_3['zem1'] - tytler2009_3['zem3']
tytler2009_3['dV13'] = (2.99e+5)*((1+tytler2009_3['zem1'])**2 - (1+tytler2009_3['zem3'])**2)/((1+tytler2009_3['zem1'])**2+(1+tytler2009_3['zem3'])**2)
tytler2009_3['Sep(kpc)13'] = tytler2009_3['Sep13']*((cosmo.arcsec_per_kpc_proper(tytler2009_3['zem1']))**(-1))

# Now adding in columns for the author+year, bibcode, and doi link
tytler2009_pairs['Paper(s)'] = "Tytler+2009"
tytler2009_pairs['bibcode(s)'] = "2009MNRAS.392.1539T"
tytler2009_pairs['DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2008.14159.x"

tytler2009_3['Paper(s)'] = "Tytler+2009"
tytler2009_3['bibcode(s)'] = "2009MNRAS.392.1539T"
tytler2009_3['DOI(s)'] = "https://doi.org/10.1111/j.1365-2966.2008.14159.x"

# Now applying a velocity difference cut of dV <= 2000 km s**-1 to ensure we are selecting systems which are \
# physically associated and not projected

tytler2009_pairs_master = (tytler2009_pairs[np.abs(tytler2009_pairs['dV'])<=2000]).reset_index(drop=True)

tytler2009_3_master = (tytler2009_3[(tytler2009_3['dV12']<=2000) & (tytler2009_3['dV13']<=2000) & (tytler2009_3['dV23']<=2000)]).reset_index(drop=True)

# Now dropping any remaining irrelevant columns: '---1', '---2', 'Pair', 'm_Pair1', 'm_Pair2', 'Sep2'
tytler2009_pairs_master.drop(labels=['---1', '---2', 'Pair', 'm_Pair1', 'm_Pair2', 'Sep2'], axis=1, inplace=True)

#tytler2009_3_master.drop(labels=['---1', '---2', 'Pair', 'm_Pair1', 'm_Pair2', 'Sep2'], axis=1, inplace=True)

# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = tytler2009_pairs_master['RA1'], dec = tytler2009_pairs_master['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_pairs_master['RA1_deg'] = coordconvert.ra.degree
tytler2009_pairs_master['Dec1_deg'] = coordconvert.dec.degree

coordconvert = SkyCoord(ra = tytler2009_pairs_master['RA2'], dec = tytler2009_pairs_master['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))

tytler2009_pairs_master['RA2_deg'] = coordconvert.ra.degree
tytler2009_pairs_master['Dec2_deg'] = coordconvert.dec.degree

# Adding a column that stores the index value, which we'll use soon to remove duplicate sources that appear in \
# both the Kirkman and Tytler catalogs
tytler2009_pairs_master['index'] = tytler2009_pairs_master.index


In [24]:
#tytler2009_pairs_master

In [25]:
#tytler2009_3_master

In [26]:
# Now we're going to work on matching the catalogs of Kirman+2008 and Tytler+2009 to remove duplicate entries

kirk = SkyCoord(ra=kirkman2008_master['RA1_deg']*u.degree, dec=kirkman2008_master['Dec1_deg']*u.degree) # Storing the coordinates from Kirkman+2008
tytler = SkyCoord(ra=tytler2009_pairs_master['RA1_deg']*u.degree, dec=tytler2009_pairs_master['Dec1_deg']*u.degree) # Storing the coordinates from Tytler+2009
idx, d2d, d3d = kirk.match_to_catalog_sky(tytler)   # Now matching the Kirkman+2008 catalog to Tytler+2009 catalog
# idx are the indices in the Tytler catalog which are the closest matching rows to the Kirkman catalog

max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
kirk_matches = kirk[sep_constraint] # Applying the filter to matches in the kirkman catalog
catalog_matches = tytler[idx[sep_constraint]] # Applying the filter to matches in the Tytler catalog

tytler_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the Tytler catalog
tytler_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
tytler_matchesRA1['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
tytler_matchesRA1['Dec1'] = catalog_matches.dec.degree

# Now, since we're not sure if the components have been mixed up between the two catalogs, we need to rematch \
# the Kirkman RA1 and Dec1 to the Tytler RA2 and Dec2

kirk = SkyCoord(ra=kirkman2008_master['RA1_deg']*u.degree, dec=kirkman2008_master['Dec1_deg']*u.degree) # Storing the coordinates from Kirkman+2008
tytler = SkyCoord(ra=tytler2009_pairs_master['RA2_deg']*u.degree, dec=tytler2009_pairs_master['Dec2_deg']*u.degree) # Storing the coordinates from Tytler+2009
idx, d2d, d3d = kirk.match_to_catalog_sky(tytler)   # Now matching the Kirkman+2008 catalog to Tytler+2009 catalog
# idx are the indices in the Tytler catalog which are the closest matching rows to the Kirkman catalog

max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
kirk_matches = kirk[sep_constraint] # Applying the filter to matches in the kirkman catalog
catalog_matches = tytler[idx[sep_constraint]] # Applying the filter to matches in the Tytler catalog

tytler_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the Tytler catalog
tytler_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
tytler_matchesRA2['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
tytler_matchesRA2['Dec1'] = catalog_matches.dec.degree

# Now we'll concatenate these two dataframes and use the resultant table to remove the matched indices from the \
# Tytler+2009 catalog. We'll defer to Kirkman+2008 for the coordinates (a completeyl arbitrary choice), and we \
# *should* just rematch all of the Kirkman and Tytler sources to Simbad and get new redshifts, since I've noticed \
# the redshifts between Kirkman+2008 and Tytler+2009 can be discrepant. 

frames = [tytler_matchesRA1,tytler_matchesRA2] 
tytler_duplicates = (pd.concat(fran vbn     mes)).reset_index(drop=True) #Concatenating them here...

# There's a total of 21 matches between the two catalogs...

# Now removing from the Tytler+2009 catalog the sources that also appear in the Kirkman+2008 catalog by removing \
# the indices listed in the tytler_duplicates table
tytler2009_master = (tytler2009_pairs_master[~tytler2009_pairs_master['index'].isin(tytler_duplicates['idx'])]).reset_index(drop=True)

#print(kirkman2008_master.columns)
#print(tytler2009_master.columns)


In [27]:
# Renaming the columns so that we have matching column names between these catalogs
kirkman2008_master.columns=['z1', 'mag', 'n_mag', 'z2', 'mag2', 'n_mag2', 'Sep', 'RA1', 'Dec1',\
                            'RA2', 'Dec2', 'Equinox', 'Coordinate_waveband', 'Coordinate_Source', \
                            'Brightness_type1', 'Brightness_type2', 'delta_z', 'dV', 'Sep(kpc)', \
                            'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg','Dec2_deg']

tytler2009_master.columns=['z1', 'Sep', 'RA1', 'Dec1', 'z2', 'RA2', 'Dec2', 'Equinox', \
                           'Coordinate_waveband', 'Coordinate_Source', 'delta_z', 'dV', 'Sep(kpc)', \
                           'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg', \
                           'Dec2_deg', 'index']        

# Dropping the index column from tytler2009
tytler2009_master.drop(labels=['index'], axis=1, inplace=True)
    
# Rearranging the columns before concatenating
kirkman2008_master = kirkman2008_master[['z1', 'Sep', 'RA1', 'Dec1', 'z2','RA2', 'Dec2', 'Equinox',\
                                         'Coordinate_waveband', 'Coordinate_Source','delta_z', 'dV', 'Sep(kpc)', \
                                         'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg', \
                                         'Dec2_deg','mag', 'n_mag','mag2', 'n_mag2','Brightness_type1', \
                                         'Brightness_type2',]]

tytler2009_master = tytler2009_master[['z1', 'Sep', 'RA1', 'Dec1', 'z2', 'RA2', 'Dec2', 'Equinox', \
                                       'Coordinate_waveband', 'Coordinate_Source', 'delta_z', 'dV', 'Sep(kpc)', \
                                       'Paper(s)', 'BibCode(s)', 'DOI(s)', 'RA1_deg', 'Dec1_deg', 'RA2_deg', \
                                       'Dec2_deg']]

# Now concatenating these frames...
frames=[kirkman2008_master,tytler2009_master]
kirktyt = (pd.concat(frames)).reset_index(drop=True) #This is now our master frame for the Kirkman+2008 and Tytler+2009 catalogs
# Duplicates at this point have been removed. 
# I have NOTE, however, noted which ones overlapped between the two using the Paper(s), BibCode(s), or DOI(s) columns
kirktyt['index'] = kirktyt.index #We'll use this to remove duplicates in a moment...


In [12]:
kirktyt

,z1,Sep,RA1,Dec1,z2,RA2,Dec2,Equinox,Coordinate_waveband,Coordinate_Source,delta_z,dV,Sep(kpc),Paper(s),BibCode(s),DOI(s),RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,mag,n_mag,mag2,n_mag2,Brightness_type1,Brightness_type2,index
0,2.995,13.70,11:16:10.7,+41:18:14.5,2.983,11:16:11.7,+41:18:21.5,J2000,NA,NA,0.012,899.471517,105.587526,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,169.044583,41.304028,169.048750,41.305972,19.5,g,18.6,g,-100,-100,0
1,1.893,15.00,9:9:23.1,+0:2:3.9,1.878,9:9:24.0,+0:2:11.0,J2000,NA,NA,0.015,1554.312840,126.226223,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,137.346250,0.034417,137.350000,0.036389,20.2,g,16.8,g,-100,-100,1
2,1.890,24.80,11:7:27.1,+0:34:7.3,1.881,11:7:25.7,+0:33:53.6,J2000,NA,NA,0.009,932.591736,208.721111,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,166.862917,0.568694,166.857083,0.564889,20.2,g,19.1,g,-100,-100,2
3,2.389,33.20,14:35:6.4,+0:9:1.5,2.384,14:35:8.3,+0:8:44.4,J2000,NA,NA,0.005,441.458493,270.502455,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,218.776667,0.150417,218.784583,0.145667,20.0,bJ,20.1,bJ,-100,-100,3
4,1.932,49.50,23:9:11.9,-27:32:27.1,1.927,23:9:15.3,-27:32:45.3,J2000,NA,NA,0.005,510.325623,415.806008,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,347.299583,-27.540861,347.313750,-27.545917,19.4,bJ,20.2,bJ,-100,-100,4
5,3.242,61.50,21:42:25.9,-44:20:17.0,3.227,21:42:22.2,-44:19:30.0,J2000,NA,NA,0.015,1059.153603,462.373980,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,325.607917,-44.338056,325.592500,-44.325000,18.7,V,21.2,V,-100,-100,5
6,2.995,68.20,10:16:5.8,+40:40:5.8,2.984,10:16:1.5,+40:40:52.9,J2000,NA,NA,0.011,824.412520,525.625493,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,154.024167,40.668278,154.006250,40.681361,20.5,g,19.6,g,-100,-100,6
7,2.566,71.90,3:10:36.5,-30:51:8.4,2.546,3:10:41.1,-30:50:27.5,J2000,NA,NA,0.020,1681.651493,577.053134,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,47.652083,-30.852333,47.671250,-30.840972,20.4,bJ,19.5,bJ,-100,-100,7
8,2.480,74.00,12:12:51.1,-0:53:42.2,2.467,12:12:56.1,-0:53:36.5,J2000,NA,NA,0.013,1119.040273,598.368657,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,183.212917,-0.895056,183.233750,-0.893472,20.4,bJ,20.8,bJ,-100,-100,8
9,1.878,88.60,0:44:34.1,+0:19:3.5,1.875,0:44:39.3,+0:18:22.8,J2000,NA,NA,0.003,311.837217,746.052452,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,11.142083,0.317639,11.163750,0.306333,19.2,g,18.3,g,-100,-100,9


In [28]:
# Now we need to check or duplicates between the kirktyt master catalog and the Hennawi catalog

henn = SkyCoord(ra=hennawi2006['RA1_deg']*u.degree, dec=hennawi2006['Dec1_deg']*u.degree) # Storing the coordinates from Hennawi+2006
KT = SkyCoord(ra=kirktyt['RA1_deg']*u.degree, dec=kirktyt['Dec1_deg']*u.degree) # Storing the coordinates from kirktyt master catalog
idx, d2d, d3d = henn.match_to_catalog_sky(KT)   # Now matching the Hennawi+2006 catalog to Kirktyt master catalog
# idx are the indices in the kirktyt catalog which are the closest matching rows to the hennawi catalog

max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
henn_matches = henn[sep_constraint] # Applying the filter to matches in the hennawi catalog
catalog_matches = KT[idx[sep_constraint]] # Applying the filter to matches in the kirktyt catalog

KT_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the KT catalog
KT_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
KT_matchesRA1['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
KT_matchesRA1['Dec1'] = catalog_matches.dec.degree

###

henn = SkyCoord(ra=hennawi2006['RA1_deg']*u.degree, dec=hennawi2006['Dec1_deg']*u.degree) # Storing the coordinates from Hennawi+2006
KT = SkyCoord(ra=kirktyt['RA2_deg']*u.degree, dec=kirktyt['Dec2_deg']*u.degree) # Storing the coordinates from kirktyt master catalog
idx, d2d, d3d = henn.match_to_catalog_sky(KT)   # Now matching the Hennawi+2006 catalog to Kirktyt master catalog
# idx are the indices in the kirktyt catalog which are the closest matching rows to the hennawi catalog

max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
henn_matches = henn[sep_constraint] # Applying the filter to matches in the hennawi catalog
catalog_matches = KT[idx[sep_constraint]] # Applying the filter to matches in the kirktyt catalog

KT_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the KT catalog
KT_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
KT_matchesRA2['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
KT_matchesRA2['Dec1'] = catalog_matches.dec.degree

# Concatenating these frames now
frames = [KT_matchesRA1,KT_matchesRA2]
KT_dups = (pd.concat(frames)).reset_index(drop=True)
# There are a total of 10 matches between the Kirktyt master catalog and the hennawi+ catalog.
# We will defer to the Hennawi+ catalog, and therefore, we're dropping the rows from the Kirktyt table

# Now removing from the Tytler+2009 catalog the sources that also appear in the Kirkman+2008 catalog by removing \
# the indices listed in the tytler_duplicates table
kirktyt_master = (kirktyt[~kirktyt['index'].isin(KT_dups['idx'])]).reset_index(drop=True)
kirktyt_master.drop(labels=['index'], axis=1, inplace=True) # Now dropping the index column as it is not needed



In [29]:
kirktyt_master

,z1,Sep,RA1,Dec1,z2,RA2,Dec2,Equinox,Coordinate_waveband,Coordinate_Source,delta_z,dV,Sep(kpc),Paper(s),BibCode(s),DOI(s),RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,mag,n_mag,mag2,n_mag2,Brightness_type1,Brightness_type2
0,1.932,49.50,23:9:11.9,-27:32:27.1,1.927,23:9:15.3,-27:32:45.3,J2000,NA,NA,0.005,510.325623,415.806008,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,347.299583,-27.540861,347.313750,-27.545917,19.4,bJ,20.2,bJ,-100,-100
1,3.242,61.50,21:42:25.9,-44:20:17.0,3.227,21:42:22.2,-44:19:30.0,J2000,NA,NA,0.015,1059.153603,462.373980,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,325.607917,-44.338056,325.592500,-44.325000,18.7,V,21.2,V,-100,-100
2,2.566,71.90,3:10:36.5,-30:51:8.4,2.546,3:10:41.1,-30:50:27.5,J2000,NA,NA,0.020,1681.651493,577.053134,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,47.652083,-30.852333,47.671250,-30.840972,20.4,bJ,19.5,bJ,-100,-100
3,1.980,120.00,2:9:54.8,-10:2:23.0,1.979,2:10:0.1,-10:3:54.0,J2000,NA,NA,0.001,100.352405,1005.585751,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,32.478333,-10.039722,32.500417,-10.065000,19.6,r,19.3,g,-100,-100
4,2.192,126.10,15:48:50.2,+53:38:43.0,2.172,15:48:40.8,+53:37:8.6,J2000,NA,NA,0.020,1879.302621,1042.923514,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,237.209167,53.645278,237.170000,53.619056,19.8,g,18.8,g,-100,-100
5,2.055,149.30,11:45:53.7,-0:33:4.5,2.048,11:45:47.5,-0:31:6.7,J2000,NA,NA,0.007,685.891282,1245.852725,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,176.473750,-0.551250,176.447917,-0.518528,20.1,g,18.7,g,-100,-100
6,2.007,169.80,16:50:51.1,+34:43:10.0,1.987,16:50:43.3,+34:45:30.0,J2000,NA,NA,0.020,1995.306448,1420.831089,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,252.712917,34.719444,252.680417,34.758333,18.5,g,20.3,g,-100,-100
7,3.768,182.60,9:44:53.8,+50:43:0.0,3.748,9:45:8.0,+50:40:57.0,J2000,NA,NA,0.020,1256.825049,1300.407195,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,146.224167,50.716667,146.283333,50.682500,20.6,g,19.8,g,-100,-100
8,2.654,191.00,10:40:19.1,+32:21:56.4,2.633,10:40:4.0,+32:21:50.6,J2000,NA,NA,0.021,1723.328627,1520.822006,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,160.079583,32.365667,160.016667,32.364056,20.6,g,19.0,g,-100,-100
9,3.242,191.10,8:3:5.8,+50:32:15.3,3.239,8:3:21.3,+50:34:17.4,J2000,NA,NA,0.003,211.531633,1436.742562,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x,120.774167,50.537583,120.838750,50.571500,20.5,g,20.3,g,-100,-100


In [30]:
#Now adding in the missing columns that are needed for the Kirkman and Tytler pairs:
kirktyt_master['System Type'] = "Binary Quasar"
kirktyt_master['Name1'] = ("J"+(kirktyt_master['RA1'])+(kirktyt_master['Dec1'])).str.replace(':','') # Adopting names for the pairs using the provided RA and Decs.
kirktyt_master['Name2'] = ("J"+(kirktyt_master['RA2'])+(kirktyt_master['Dec2'])).str.replace(':','') 
kirktyt_master['z1_type'] = -100
kirktyt_master['z2_type'] = -100
kirktyt_master['Selection Method'] = -100
kirktyt_master['Confirmation Method'] = -100

# Now renaming the columns to a more uniform scheme where:
# mag and mag2 --> Brightness1 and Brightness2
# n_mag and n_mag2 --> Brightness_band1 and Brightness_band2 
kirktyt_master.columns=['z1', 'Sep', 'RA1', 'Dec1', 'z2', 'RA2', 'Dec2', 'Equinox','Coordinate_waveband',\
                        'Coordinate_Source', 'delta_z', 'dV', 'Sep(kpc)','Paper(s)', 'BibCode(s)', 'DOI(s)',\
                        'RA1_deg', 'Dec1_deg', 'RA2_deg','Dec2_deg', 'Brightness1', 'Brightness_band1',\
                        'Brightness2', 'Brightness_band2','Brightness_type1','Brightness_type2', 'System Type',\
                        'Name1', 'Name2', 'z1_type','z2_type', 'Selection Method', 'Confirmation Method']

# Now rearranging the columns before we concatenate with the Hennawi+2006 catalog...

kirktyt_master = kirktyt_master[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1',\
                         'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg',\
                         'Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
                         'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
                         'Brightness_type2','Sep','Sep(kpc)','delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]


# NOTE: I NEED TO COME BACK AND FIX THE RA AND DEC VALUES SO THAT THE NAMING IS ACTUALLY CORRECT, RATHER THAN \
# CONTINUING TO HAVE (E.G.) 01HR BE LISTED AS 1:... AND NOT 01:...

In [55]:
kirktyt_master

,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Sep(kpc),delta_z,dV,Paper(s),BibCode(s),DOI(s)
0,Binary Quasar,J23911.9-273227.1,J23915.3-273245.3,-100,-100,1.932,-100,1.927,-100,23:9:11.9,-27:32:27.1,347.299583,-27.540861,23:9:15.3,-27:32:45.3,347.313750,-27.545917,J2000,NA,NA,19.4,bJ,-100,20.2,bJ,-100,49.50,415.806008,0.005,510.325623,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
1,Binary Quasar,J214225.9-442017.0,J214222.2-441930.0,-100,-100,3.242,-100,3.227,-100,21:42:25.9,-44:20:17.0,325.607917,-44.338056,21:42:22.2,-44:19:30.0,325.592500,-44.325000,J2000,NA,NA,18.7,V,-100,21.2,V,-100,61.50,462.373980,0.015,1059.153603,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
2,Binary Quasar,J31036.5-30518.4,J31041.1-305027.5,-100,-100,2.566,-100,2.546,-100,3:10:36.5,-30:51:8.4,47.652083,-30.852333,3:10:41.1,-30:50:27.5,47.671250,-30.840972,J2000,NA,NA,20.4,bJ,-100,19.5,bJ,-100,71.90,577.053134,0.020,1681.651493,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
3,Binary Quasar,J2954.8-10223.0,J2100.1-10354.0,-100,-100,1.980,-100,1.979,-100,2:9:54.8,-10:2:23.0,32.478333,-10.039722,2:10:0.1,-10:3:54.0,32.500417,-10.065000,J2000,NA,NA,19.6,r,-100,19.3,g,-100,120.00,1005.585751,0.001,100.352405,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
4,Binary Quasar,J154850.2+533843.0,J154840.8+53378.6,-100,-100,2.192,-100,2.172,-100,15:48:50.2,+53:38:43.0,237.209167,53.645278,15:48:40.8,+53:37:8.6,237.170000,53.619056,J2000,NA,NA,19.8,g,-100,18.8,g,-100,126.10,1042.923514,0.020,1879.302621,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
5,Binary Quasar,J114553.7-0334.5,J114547.5-0316.7,-100,-100,2.055,-100,2.048,-100,11:45:53.7,-0:33:4.5,176.473750,-0.551250,11:45:47.5,-0:31:6.7,176.447917,-0.518528,J2000,NA,NA,20.1,g,-100,18.7,g,-100,149.30,1245.852725,0.007,685.891282,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
6,Binary Quasar,J165051.1+344310.0,J165043.3+344530.0,-100,-100,2.007,-100,1.987,-100,16:50:51.1,+34:43:10.0,252.712917,34.719444,16:50:43.3,+34:45:30.0,252.680417,34.758333,J2000,NA,NA,18.5,g,-100,20.3,g,-100,169.80,1420.831089,0.020,1995.306448,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
7,Binary Quasar,J94453.8+50430.0,J9458.0+504057.0,-100,-100,3.768,-100,3.748,-100,9:44:53.8,+50:43:0.0,146.224167,50.716667,9:45:8.0,+50:40:57.0,146.283333,50.682500,J2000,NA,NA,20.6,g,-100,19.8,g,-100,182.60,1300.407195,0.020,1256.825049,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
8,Binary Quasar,J104019.1+322156.4,J10404.0+322150.6,-100,-100,2.654,-100,2.633,-100,10:40:19.1,+32:21:56.4,160.079583,32.365667,10:40:4.0,+32:21:50.6,160.016667,32.364056,J2000,NA,NA,20.6,g,-100,19.0,g,-100,191.00,1520.822006,0.021,1723.328627,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x
9,Binary Quasar,J835.8+503215.3,J8321.3+503417.4,-100,-100,3.242,-100,3.239,-100,8:3:5.8,+50:32:15.3,120.774167,50.537583,8:3:21.3,+50:34:17.4,120.838750,50.571500,J2000,NA,NA,20.5,g,-100,20.3,g,-100,191.10,1436.742562,0.003,211.531633,Kirkman+2008,2008MNRAS.391.1457K,https://doi.org/10.1111/j.1365-2966.2008.13994.x


In [37]:
print(hennawi2006.columns)
print(kirktyt_master.columns)

Index(['System Type', 'Name1', 'Name2', 'Selection Method',
       'Confirmation Method', 'z1', 'z1_type', 'z2', 'z2_type', 'RA1', 'Dec1',
       'RA1_deg', 'Dec1_deg', 'RA2', 'Dec2', 'RA2_deg', 'Dec2_deg', 'Equinox',
       'Coordinate_waveband', 'Coordinate_Source', 'Brightness1',
       'Brightness_band1', 'Brightness_type1', 'Brightness2',
       'Brightness_band2', 'Brightness_type2', 'Sep', 'Sep(kpc)', 'delta_z',
       'dV', 'Paper(s)', 'BibCode(s)', 'DOI(s)'],
      dtype='object')
Index(['System Type', 'Name1', 'Name2', 'Selection Method',
       'Confirmation Method', 'z1', 'z1_type', 'z2', 'z2_type', 'RA1', 'Dec1',
       'RA1_deg', 'Dec1_deg', 'RA2', 'Dec2', 'RA2_deg', 'Dec2_deg', 'Equinox',
       'Coordinate_waveband', 'Coordinate_Source', 'Brightness1',
       'Brightness_band1', 'Brightness_type1', 'Brightness2',
       'Brightness_band2', 'Brightness_type2', 'Sep', 'Sep(kpc)', 'delta_z',
       'dV', 'Paper(s)', 'BibCode(s)', 'DOI(s)'],
      dtype='object')


In [31]:
# Now that the columns from Hennawi+2006 and the Kirkman+Tytler catalogs have common names and ordering (and \
# all irrelevant columns and duplicates have been dropped), we will concatenate these tables:

frames=[hennawi2006,kirktyt_master]
hennkirktyt = (pd.concat(frames)).reset_index(drop=True) # Concatenating the tables here


In [32]:
#hennkirktyt

In [33]:
# This cell is simply checking to ensure that there are no common rows between the Wang+2009 double-peak-selected \
# targets and the targets assembled in the Hennawi/Kirkman/Tytler catalogs. There are no matches!

HKT = SkyCoord(ra=hennkirktyt['RA1_deg']*u.degree, dec=hennkirktyt['Dec1_deg']*u.degree) # Storing the coordinates from hennkirktyt
wang = SkyCoord(ra=wang2009['RA1_deg']*u.degree, dec=wang2009['Dec1_deg']*u.degree) # Storing the coordinates from wang2009 master catalog
idx, d2d, d3d = HKT.match_to_catalog_sky(wang)   # Now matching the hennkirktyt catalog to wang2009 master catalog
# idx are the indices in the wang2009 catalog which are the closest matching rows to the hennkirktyt catalog

max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
HKT_matches = HKT[sep_constraint] # Applying the filter to matches in the hennkirktyt catalog
catalog_matches = wang[idx[sep_constraint]] # Applying the filter to matches in the wang2009 catalog

wang_matchesRA1 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the wang catalog
wang_matchesRA1['idx'] = idx[sep_constraint] # Storing the indices
wang_matchesRA1['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
wang_matchesRA1['Dec1'] = catalog_matches.dec.degree

###

HKT = SkyCoord(ra=hennkirktyt['RA2_deg']*u.degree, dec=hennkirktyt['Dec2_deg']*u.degree) # Storing the coordinates from hennkirktyt
wang = SkyCoord(ra=wang2009['RA1_deg']*u.degree, dec=wang2009['Dec1_deg']*u.degree) # Storing the coordinates from wang2009 master catalog
idx, d2d, d3d = HKT.match_to_catalog_sky(wang)   # Now matching the hennkirktyt catalog to wang2009 master catalog
# idx are the indices in the wang2009 catalog which are the closest matching rows to the hennkirktyt catalog

max_sep = 5.0 * u.arcsec # The max match tolerance will be 5''
sep_constraint = d2d <= max_sep # Filtering the angular separations from the matches to ≤ 5''
HKT_matches = HKT[sep_constraint] # Applying the filter to matches in the hennkirktyt catalog
catalog_matches = wang[idx[sep_constraint]] # Applying the filter to matches in the wang2009 catalog

wang_matchesRA2 = pd.DataFrame() # Creating an empty dataframe to store the indices and coordinates of the matches in the wang catalog
wang_matchesRA2['idx'] = idx[sep_constraint] # Storing the indices
wang_matchesRA2['RA1'] = catalog_matches.ra.degree # Storing the coordinates in these two rows
wang_matchesRA2['Dec1'] = catalog_matches.dec.degree

# Concatenating these frames now
frames = [wang_matchesRA1,wang_matchesRA2]
wang_dups = (pd.concat(frames)).reset_index(drop=True)

wang_dups

,idx,RA1,Dec1


In [34]:
# Now concatenating the hennkirktyt and the wang catalog
frames=[wang2009,hennkirktyt]
HKTW = pd.concat(frames).reset_index(drop=True)


In [119]:
HKTW

,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Sep(kpc),delta_z,dV,Paper(s),BibCode(s),DOI(s)
0,Dual AGN Candidate,J000249+004504,J000249+004504,Double-Peaked Optical Spectroscopic Emission L...,-99,0.08662,spec,0.08662,spec,00:02:49,+00:45:04,0.704167,0.751111,00:02:49,+00:45:04,0.704167,0.751111,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
1,Dual AGN Candidate,J000656+154847,J000656+154847,Double-Peaked Optical Spectroscopic Emission L...,-99,0.12515,spec,0.12515,spec,00:06:56,+15:48:47,1.733333,15.813056,00:06:56,+15:48:47,1.733333,15.813056,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
2,Dual AGN Candidate,J013555+143529,J013555+143529,Double-Peaked Optical Spectroscopic Emission L...,-99,0.07208,spec,0.07208,spec,01:35:55,+14:35:29,23.979167,14.591389,01:35:55,+14:35:29,23.979167,14.591389,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
3,Dual AGN Candidate,J014209-005049,J014209-005049,Double-Peaked Optical Spectroscopic Emission L...,-99,0.13253,spec,0.13253,spec,01:42:09,-00:50:49,25.537500,-0.846944,01:42:09,-00:50:49,25.537500,-0.846944,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
4,Dual AGN Candidate,J015605-000721,J015605-000721,Double-Peaked Optical Spectroscopic Emission L...,-99,0.08084,spec,0.08084,spec,01:56:05,-00:07:21,29.020833,-0.122500,01:56:05,-00:07:21,29.020833,-0.122500,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
5,Dual AGN Candidate,J073509+403624,J073509+403624,Double-Peaked Optical Spectroscopic Emission L...,-99,0.10297,spec,0.10297,spec,07:35:09,+40:36:24,113.787500,40.606667,07:35:09,+40:36:24,113.787500,40.606667,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
6,Dual AGN Candidate,J074729+344018,J074729+344018,Double-Peaked Optical Spectroscopic Emission L...,-99,0.12982,spec,0.12982,spec,07:47:29,+34:40:18,116.870833,34.671667,07:47:29,+34:40:18,116.870833,34.671667,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
7,Dual AGN Candidate,J074953+451454,J074953+451454,Double-Peaked Optical Spectroscopic Emission L...,-99,0.03132,spec,0.03132,spec,07:49:53,+45:14:54,117.470833,45.248333,07:49:53,+45:14:54,117.470833,45.248333,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
8,Dual AGN Candidate,J075223+273643,J075223+273643,Double-Peaked Optical Spectroscopic Emission L...,-99,0.06908,spec,0.06908,spec,07:52:23,+27:36:43,118.095833,27.611944,07:52:23,+27:36:43,118.095833,27.611944,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009ApJ...705L..76W,https://doi.org/10.1088/0004-637X/705/1/L76
9,Dual AGN Candidate,J080218+304622,J080218+304622,Double-Peaked Optical Spectroscopic Emission L...,-99,0.07654,spec,0.07654,spec,08:02:18,+30:46:22,120.575000,30.772778,08:02:18,+30:46:22,120.575000,30.772778,J2000,Optical,SDSS,-100.00,-100,-100,-100.00,-100,-100,-1.00,-1.000000,-99.000,-99.000000,Wang+2009,2009A

In [6]:
# Now loading in Gattano+2014 here

gattano2014 = ((Table.read('Tables/Gattano2014/table7.dat', readme = 'Tables/Gattano2014/ReadMe', format='ascii.cds')).to_pandas()) 

gattano2014['Name'] = gattano2014['QSO1']
gattano2014['Name2'] = gattano2014['QSO2']
#gattano2014['z2'] = gattano2014['z2']
#gattano2014['z1_type'] = "spec"
#gattano2014['z2_type'] = "spec"

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(gattano2014,gattano2014['Name'])

## Converting the coordinates
#coordconvert = SkyCoord(ra = gattano2014['RA'], dec = gattano2014['Dec'], frame='icrs', unit = (u.hourangle, u.deg))
#gattano2014['RA1_deg'] = coordconvert.ra.degree
#gattano2014['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#gattano2014['RA2'] = gattano2014['RA']
#gattano2014['Dec2'] = gattano2014['Dec']

#gattano2014['RA2_deg'] = gattano2014['RA1_deg']
#gattano2014['Dec2_deg'] = gattano2014['Dec1_deg']

# Adding details about the coordinates
gattano2014['Equinox'] = "J2000"
gattano2014['Coordinate_waveband'] = "Optical"
#gattano2014['Coordinate_Source'] = "SDSS"

gattano2014['System Type'] = 'Dual AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
gattano2014['Brightness1'] = -100
gattano2014['Brightness_band1'] = -100
gattano2014['Brightness_type1'] = -100

gattano2014['Brightness2'] = -100
gattano2014['Brightness_band2'] = -100
gattano2014['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
gattano2014['Sep'] = gattano2014['theta'] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#gattano2014['Sep(kpc)'] = gattano2014['Sep']*((cosmo.arcsec_per_kpc_proper(gattano2014['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#gattano2014['delta_z'] = gattano2014['z']-gattano2014['z2']
gattano2014['dV'] = (2.99e+5)*((1+gattano2014['z1'])**2 - (1+gattano2014['z2'])**2)/((1+gattano2014['z1'])**2+(1+gattano2014['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
gattano2014['Selection Method'] = "Complex. Needs review." #DPSELs
gattano2014['Confirmation Method'] = "-99"
gattano2014['Paper(s)'] = "Gattano+2014"
gattano2014['BibCode(s)'] = "2014A&A...564A.117G"
gattano2014['DOI(s)'] = "https://doi.org/10.1051/0004-6361/201323238"

gattano2014['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#gattano2014.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#gattano2014 = gattano2014[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#gattano2014.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']

#gattano2014


gattano2014


,---,QSO1/2,U1mag,U2mag,theta,z1,z2,Ref1,Ref2,SeqU,Seqz,QSO1,---_1,QSO2,Name,Name2,Equinox,Coordinate_waveband,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,dV,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s),Notes
0,LQAC_,250+031_012/013,19.63,0.00,2.1322,0.587,0.586,----E-----KLM,------------M,96,112,250+031_012,/,250+031_013,250+031_012,250+031_013,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.1322,188.465156,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
1,LQAC_,238+046_013/014,0.00,0.00,2.3773,0.136,0.135,------------M,------------M,143,144,238+046_013,/,238+046_014,238+046_013,238+046_014,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.3773,263.320072,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
2,LQAC_,209+018_002/003,0.00,14.91,2.4501,0.051,0.051,------------M,------------M,2,157,209+018_002,/,209+018_003,209+018_002,209+018_003,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4501,0.000000,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
3,LQAC_,247+037_006/007,18.92,19.40,2.4523,0.923,0.922,------------M,----E-----K-M,49,101,247+037_006,/,247+037_007,247+037_006,247+037_007,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.4523,155.526647,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
4,LQAC_,181+043_009/010,18.26,18.65,2.6083,1.789,1.791,------------M,----E-------M,30,59,181+043_009,/,181+043_010,181+043_009,181+043_010,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.6083,-214.336890,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
5,LQAC_,126+003_006/007,17.33,0.00,2.7771,0.506,0.506,ABCD-----J--M,--------I-KL-,9,120,126+003_006,/,126+003_007,126+003_006,126+003_007,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.7771,0.000000,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
6,LQAC_,242+028_012/013,19.66,20.84,2.8096,0.170,0.170,------------M,----E----J--M,100,139,242+028_012,/,242+028_013,242+028_012,242+028_013,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.8096,0.000000,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
7,LQAC_,052-000_030/031,24.52,19.53,2.8899,0.610,0.609,----E--------,----E-----KL-,88,107,052-000_030,/,052-000_031,052-000_030,052-000_031,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.8899,185.771961,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
8,LQAC_,123+040_002/003,22.26,18.30,2.9534,0.188,0.189,----E--------,----E----JKLM,31,137,123+040_002,/,123+040_003,123+040_002,123+040_003,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,2.9534,-251.577574,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,
9,LQAC_,179+012_006/007,19.53,19.82,3.3223,0.596,0.596,------------M,------------M,87,109,179+012_006,/,179+012_007,179+012_006,179+012_007,J2000,Optical,Dual AGN Candidate,-100,-100,-100,-100,-100,-100,3.3223,0.000000,Complex. Needs review.,-99,Gattano+2014,2014A&A...564A.117G,https://doi.org/10.1051/0004-6361/201323238,


In [15]:
# Now loading in the newly discovered 54 binary quasars presented in Findlay+2018:

findlay2018 = pd.read_csv('Tables/Findlay2018/Findlay_2018.csv', sep=',')

findlay2018['Name1'] = findlay2018['QSO_fg']
findlay2018['Name2'] = findlay2018['QSO_bg']

findlay2018['z1'] = findlay2018['z_fg']
findlay2018['z2'] = findlay2018['z_bg']
findlay2018['z1_type'] = "spec"
findlay2018['z2_type'] = "spec"



# Converting the coordinates
coordconvert = SkyCoord(ra = findlay2018['RA1'], dec = findlay2018['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
findlay2018['RA1_deg'] = coordconvert.ra.degree
findlay2018['Dec1_deg'] = coordconvert.dec.degree

# Adding in a second set of coordinates for the 'secondary'
#findlay2018['RA2'] = findlay2018['RA']
#findlay2018['Dec2'] = findlay2018['Dec']

coordconvert = SkyCoord(ra = findlay2018['RA2'], dec = findlay2018['Dec2'], frame='icrs', unit = (u.hourangle, u.deg))
findlay2018['RA2_deg'] = coordconvert.ra.degree
findlay2018['Dec2_deg'] = coordconvert.dec.degree

# Adding details about the coordinates
findlay2018['Equinox'] = "J2000"
findlay2018['Coordinate_waveband'] = "Optical"
findlay2018['Coordinate_Source'] = "SDSS"

findlay2018['System Type'] = 'Dual AGN'

# Adding in some columns that we'll population via a Simbad or Ned search later
findlay2018['Brightness1'] = -100
findlay2018['Brightness_band1'] = -100
findlay2018['Brightness_type1'] = -100

findlay2018['Brightness2'] = -100
findlay2018['Brightness_band2'] = -100
findlay2018['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
findlay2018['Sep'] = findlay2018['theta\''] # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#findlay2018['Sep(kpc)'] = findlay2018['Sep']*((cosmo.arcsec_per_kpc_proper(findlay2018['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#findlay2018['delta_z'] = findlay2018['z']-findlay2018['z2']
#findlay2018['dV'] = (2.99e+5)*((1+findlay2018['z'])**2 - (1+findlay2018['z2'])**2)/((1+findlay2018['z'])**2+(1+findlay2018['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
findlay2018['Selection Method'] = "Check paper / Optical Colors / Mid-IR Colors" #DPSELs
findlay2018['Confirmation Method'] = "Optical Spectroscopy"
findlay2018['Paper(s)'] = "Findlay+2018"
findlay2018['BibCode(s)'] = "2018ApJS..236...44F"
findlay2018['DOI(s)'] = "https://doi.org/10.3847/1538-4365/aabee5"

#findlay2018['Notes'] = ''
# Here we're making manual adjustments based on the findings of Liu+2010b:


# And dropping any columns that we don't need....
#findlay2018.drop(labels=['SDSS','Plate','Fiber','MJD','f_SDSS','sigma','FWHM1','FWHM2','VOIII1','VOIII2','VHb1','VHb2'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#findlay2018 = findlay2018[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#findlay2018.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']




findlay2018




,QSO_fg,RA1,Dec1,QSO_bg,RA2,Dec2,z_fg,z_bg,theta',R_bottom(pkpc),Survey,Name1,Name2,z1,z2,z1_type,z2_type,RA1_deg,Dec1_deg,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,J003308.63-083222.19,00:33:08.63,-08:32:22.19,J003307.31-083241.55,00:33:07.31,-08:32:41.55,3.038,3.043,0.459,216.433,SDSS,J003308.63-083222.19,J003307.31-083241.55,3.038,3.043,spec,spec,8.285958,-8.539497,8.280458,-8.544875,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.459,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
1,J012902.78+191824.46,01:29:02.78,+19:18:24.46,J012901.92+191847.18,01:29:01.92,+19:18:47.18,2.680,2.691,0.430,209.778,SDSS,J012902.78+191824.46,J012901.92+191847.18,2.680,2.691,spec,spec,22.261583,19.306794,22.258000,19.313106,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.430,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
2,J015415.22+032455.84,01:54:15.22,+03:24:55.84,J015416.43+032457.86,01:54:16.43,+03:24:57.86,2.660,3.219,0.304,148.640,SDSS,J015415.22+032455.84,J015416.43+032457.86,2.660,3.219,spec,spec,28.563417,3.415511,28.568458,3.416072,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.304,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
3,J022845.72-124643.92,02:28:45.72,-12:46:43.92,J022848.07-124706.78,02:28:48.07,-12:47:06.78,1.733,2.032,0.688,358.303,ATLAS,J022845.72-124643.92,J022848.07-124706.78,1.733,2.032,spec,spec,37.190500,-12.778867,37.200292,-12.785217,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.688,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
4,J023229.05-100123.48,02:32:29.05,-10:01:23.48,J023231.25-100102.92,02:32:31.25,-10:01:02.92,2.063,2.386,0.641,328.856,ATLAS,J023229.05-100123.48,J023231.25-100102.92,2.063,2.386,spec,spec,38.121042,-10.023189,38.130208,-10.017478,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.641,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
5,J031855.31-103040.30,03:18:55.31,-10:30:40.30,J031853.87-102945.32,03:18:53.87,-10:29:45.32,2.226,2.417,0.982,498.485,ATLAS,J031855.31-103040.30,J031853.87-102945.32,2.226,2.417,spec,spec,49.730458,-10.511194,49.724458,-10.495922,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.982,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
6,J032926.40-134732.22,03:29:26.40,-13:47:32.22,J032926.04-134831.51,03:29:26.04,-13:48:31.51,2.073,2.372,0.992,508.701,ATLAS,J032926.40-134732.22,J032926.04-134831.51,2.073,2.372,spec,spec,52.360000,-13.792283,52.358500,-13.808753,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.992,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
7,J033347.40-133928.44,03:33:47.40,-13:39:28.44,J033345.40-133938.41,03:33:45.40,-13:39:38.41,2.230,2.679,0.513,260.495,ATLAS,J033347.40-133928.44,J033345.40-133938.41,2.230,2.679,spec,spec,53.447500,-13.657900,53.439167,-13.660669,J2000,Optical,SDSS,Dual AGN,-100,-100,-100,-100,-100,-100,0.513,Check paper / Optical Colors / Mid-IR Colors,Optical Spectroscopy,Findlay+2018,2018ApJS..236...44F,https://doi.org/10.3847/1538-4365/aabee5
8,J034952.34-110620.59,03:49:52.34,-11:06:20.59,J034955.77-110642.91,03:49:55.77,-11:06:42.91,2.449,2.824,0.920,458.622,ATLAS,J034952.34-110620.59,J034955.77-110642.91

In [ ]:
# Now loading

In [23]:
# Now loading in the catalog(s) from Tytler+2009
cheung2007 = (Table.read('Tables/Cheung_2007/table2.dat', readme = 'Tables/Cheung_2007/ReadMe', format='ascii.cds')).to_pandas()

cheung2007['System Type'] = "Recoil Candidate"
cheung2007['Name1'] = cheung2007['[C2007]']
cheung2007['Name2'] = cheung2007['[C2007]']
cheung2007['Selection Method'] = "Wing or X-ray Shaped Radio Source Candidate"
cheung2007['Confirmation Method'] = -99

cheung2007['z1'] = cheung2007['z']
cheung2007['z1_type'] = -100
cheung2007['z2'] = cheung2007['z']
cheung2007['z2_type'] = -100

# Creating an RA and Dec column for each source (and a duplicate column for each for the 'secondary' in each)
cheung2007['RA1'] = cheung2007['RAh'].astype(str) + ':' + cheung2007['RAm'].astype(str) + ':' + cheung2007['RAs'].astype(str)
cheung2007['Dec1'] = cheung2007['DE-'].astype(str) + cheung2007['DEd'].astype(str) + ':' + cheung2007['DEm'].astype(str) + ':' + cheung2007['DEs'].astype(str)

cheung2007['RA2'] = cheung2007['RA1']
cheung2007['Dec2'] = cheung2007['Dec1']

# And converting sexagesimal to degrees...
coordconvert = SkyCoord(ra = cheung2007['RA1'], dec = cheung2007['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
cheung2007['RA1_deg'] = coordconvert.ra.degree
cheung2007['Dec1_deg'] = coordconvert.dec.degree

cheung2007['RA2_deg'] = cheung2007['RA1_deg']
cheung2007['Dec2_deg'] = cheung2007['Dec1_deg']

# Dropping the original 7 columns for RA and Dec
cheung2007.drop(labels=['RAh','RAm','RAs','DE-','DEd','DEm','DEs'], axis=1, inplace=True)

# Dropping irrelevant columns that (as far as I know) we do not need...
cheung2007.drop(labels=['Seq','g-r','f_g-r','f_rmag','ID','S.365','S4.9','r_S4.9','alpha1','alpha2','OCat'], axis=1, inplace=True)
#'S1.4',

# Adding in columns for information about the coordinates
cheung2007['Equinox'] = "J2000"
cheung2007['Optical'] = 'Optical'
cheung2007['Coordinate_waveband'] = cheung2007['Optical'].where(((cheung2007['r_rmag']=='SDSS') | (cheung2007['r_rmag']=='APM') | (cheung2007['r_rmag']=='D89') | (cheung2007['r_rmag']=='USNO')), other='Radio')
cheung2007['Coordinate_Source'] = cheung2007['r_rmag'] # I believe this should be Cheung's column 'r_rmag'
cheung2007.drop(labels='Optical', axis=1, inplace=True)

cheung2007['Brightness1'] = cheung2007['rmag'].where(((cheung2007['rmag']>0) & (cheung2007['r_rmag']=='SDSS')), other=cheung2007['S1.4'])
cheung2007['Brightness2'] = cheung2007['Brightness1']
cheung2007['r SDSS'] = 'r SDSS'
cheung2007['Brightness_band1'] = cheung2007["r SDSS"].where(((cheung2007['r_rmag']=='SDSS') & (cheung2007['rmag']>0)), other='1.4 GHz NVSS')
cheung2007['Brightness_band2'] = cheung2007['Brightness2']
cheung2007.drop(labels='r SDSS', axis=1, inplace=True)

cheung2007['asinh model mag'] = 'asinh model mag'
cheung2007['Brightness_type1'] = cheung2007['asinh model mag'].where((cheung2007['r_rmag']=='SDSS'), other='Flux: mJy')
cheung2007['Brightness_type2'] = cheung2007['Brightness_type1']
cheung2007.drop(labels='asinh model mag', axis=1, inplace=True)

cheung2007['Sep'] = 0
cheung2007['Sep(kpc)'] = 0
cheung2007['delta_z'] = cheung2007['z1']-cheung2007['z2']
cheung2007['dV'] = (2.99e+5)*((1+cheung2007['z1'])**2 - (1+cheung2007['z2'])**2)/((1+cheung2007['z1'])**2+(1+cheung2007['z2'])**2)

# Adding in the info for Cheung's paper here
cheung2007['Paper(s)'] = "Cheung+2007"
cheung2007['BibCode(s)'] = "2007AJ....133.2097C"
cheung2007['DOI(s)'] = "https://doi.org/10.1086/513095"

# Now dropping any irrelevant columns 
cheung2007.drop(labels=['[C2007]','rmag','r_rmag','z','r_z'], axis=1, inplace=True)

# Now rearranging the columns (and renaming if need be):

cheung2007 = cheung2007[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1', \
                         'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2',\
                         'RA2_deg','Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
                         'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
                         'Brightness_type2','Sep','Sep(kpc)','delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]

In [36]:
#print(cheung2007.columns)

In [24]:
#cheung2007

In [26]:
# Now reading in the information from table 1 of Cheung+2007, which was a literature compilation of X-shaped \
# radio sources

cheung2007_t1 = pd.read_csv('Tables/Cheung_2007/Cheung2007_table1.csv', sep=',')

cheung2007_t1['Name1'] = cheung2007_t1['System Name']
cheung2007_t1['Name2'] = cheung2007_t1['System Name']
cheung2007_t1['Selection Method'] = "Wing or X-Shaped Radio Source"
cheung2007_t1['Confirmation Method'] = -99
cheung2007_t1['z1'] = cheung2007_t1['Redshift']
cheung2007_t1['z1_type'] = -100
cheung2007_t1['z2'] = cheung2007_t1['Redshift']
cheung2007_t1['z2_type'] = -100

cheung2007_t1['RA1'] = cheung2007_t1['RA']
cheung2007_t1['Dec1']= cheung2007_t1['Dec']

cheung2007_t1['RA2'] = cheung2007_t1['RA']
cheung2007_t1['Dec2']= cheung2007_t1['Dec']

# Converting now the sexagesimal coordinates to degrees...
coordconvert = SkyCoord(ra = cheung2007_t1['RA1'], dec = cheung2007_t1['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))
cheung2007_t1['RA1_deg'] = coordconvert.ra.degree
cheung2007_t1['Dec1_deg'] = coordconvert.dec.degree

cheung2007_t1['RA2_deg'] = cheung2007_t1['RA1_deg']
cheung2007_t1['Dec2_deg'] = cheung2007_t1['Dec1_deg']

cheung2007_t1['Brightness_band1'] = cheung2007_t1['Brightness_band1'].astype(str) + " (Simbad)"
cheung2007_t1['Brightness_type1'] = cheung2007_t1['Brightness_type1'].astype(str) + " (Check Simbad)"

cheung2007_t1['Brightness2'] = cheung2007_t1['Brightness1']
cheung2007_t1['Brightness_band2'] = cheung2007_t1['Brightness_band1']
cheung2007_t1['Brightness_type2'] = cheung2007_t1['Brightness_type1']

cheung2007_t1['delta_z'] = cheung2007_t1['z1']-cheung2007_t1['z2']
cheung2007_t1['dV'] = (2.99e+5)*((1+cheung2007_t1['z1'])**2 - (1+cheung2007_t1['z2'])**2)/((1+cheung2007_t1['z1'])**2+(1+cheung2007_t1['z2'])**2)


# Dropping unnecessary columns:
cheung2007_t1.drop(labels=['Component Name','J2000 Designation', 'System Name', 'Discovery Method', 'Redshift',\
                           'Redshift Type', 'RA', 'Dec'], axis=1, inplace=True)

# Tacking on Cheung+2007's paper info, just so I can note that Cheung+2007 lists all of these...
cheung2007_t1['Papers(s)'] = (cheung2007_t1['Paper(s)']).astype(str)+" ; Cheung+2007"
cheung2007_t1['BibCode(s)'] = cheung2007_t1['BibCode(s)'].astype(str)+" ; 2007AJ....133.2097C"
cheung2007_t1['DOI(s)'] = cheung2007_t1['DOI(s)'].astype(str)+" ; https://doi.org/10.1086/513095"

# Now rearranging the columns and renaming (if needed):

cheung2007_t1 = cheung2007_t1[['System Type','Name1','Name2','Selection Method','Confirmation Method','z1',\
                               'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg',\
                               'Dec2_deg','Equinox','Coordinate Waveband','Coordinate Source','Brightness1',\
                               'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
                               'Brightness_type2','Sep(arcsec)','Sep(kpc)','delta_z','dV','Paper(s)',\
                               'BibCode(s)','DOI(s)', 'Notes']]
 
cheung2007_t1.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1',\
                               'z1_type','z2','z2_type','RA1','Dec1','RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg',\
                               'Dec2_deg','Equinox','Coordinate_waveband','Coordinate_Source','Brightness1',\
                               'Brightness_band1','Brightness_type1','Brightness2','Brightness_band2',\
                               'Brightness_type2','Sep','Sep(kpc)','delta_z','dV','Paper(s)',\
                               'BibCode(s)','DOI(s)', 'Notes']

In [27]:
print(cheung2007_t1.columns)

Index(['System Type', 'Name1', 'Name2', 'Selection Method',
       'Confirmation Method', 'z1', 'z1_type', 'z2', 'z2_type', 'RA1', 'Dec1',
       'RA1_deg', 'Dec1_deg', 'RA2', 'Dec2', 'RA2_deg', 'Dec2_deg', 'Equinox',
       'Coordinate_waveband', 'Coordinate_Source', 'Brightness1',
       'Brightness_band1', 'Brightness_type1', 'Brightness2',
       'Brightness_band2', 'Brightness_type2', 'Sep', 'Sep(kpc)', 'delta_z',
       'dV', 'Paper(s)', 'BibCode(s)', 'DOI(s)', 'Notes'],
      dtype='object')


In [110]:
# We need to add 3C 293 (Liu+2004) and for J0116 at RA and Dec 01 16 25.071 -47 22 40.67

In [28]:
#cheung2007_t1

In [33]:
# Now concatenating the two tables form Cheung+2007
frames=[cheung2007_t1,cheung2007]
cheung_master = pd.concat(frames).reset_index(drop=True)


In [30]:
#cheung_master

In [11]:
# Here we're loading in the X-shaped radio sources from Proctor+2011

proctor2011 = (Table.read('Tables/Proctor2011/table8.dat', readme = 'Tables/Proctor2011/ReadMe', format='ascii.cds')).to_pandas()

# There is some overlap between Proctor+2011 and Cheung+2007. Need to match those!


proctor2011['Name1'] = proctor2011['FCG']
proctor2011['Name2'] = proctor2011['FCG']
#proctor2011['z1'] = proctor2011['z']
#proctor2011['z2'] = proctor2011['z']
#proctor2011['z1_type'] = proctor2011['n_z']
#proctor2011['z2_type'] = proctor2011['n_z']

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(yang2019,proctor2011['Designation'])

proctor2011['RA1'] = proctor2011['RAh'].astype(str) + ':' + proctor2011['RAm'].astype(str) + ':' + proctor2011['RAs'].astype(str)
proctor2011['Dec1'] = proctor2011['DE-'].astype(str) + proctor2011['DEd'].astype(str) + ':' + proctor2011['DEm'].astype(str) + ':' + proctor2011['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = proctor2011['RA1'], dec = proctor2011['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

proctor2011['RA1_deg'] = coordconvert.ra.degree
proctor2011['Dec1_deg'] = coordconvert.dec.degree

#proctor2011['Coordinates'] = proctor2011['SDSSID']#.str.slice(start=1) # Stripping the J
#proctor2011['RA_test'] = proctor2011['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
#proctor2011['Dec_test'] = proctor2011['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
#proctor2011['RA'] = proctor2011['RA_test'].str.slice(start=0, stop=2)+":"+proctor2011['RA_test'].str.slice(start=2, stop=4)+":"+proctor2011['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#proctor2011['Dec'] = proctor2011['Dec_test'].str.slice(start=0, stop=3)+":"+proctor2011['Dec_test'].str.slice(start=3, stop=5)+":"+proctor2011['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#yang2019.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Adding in a second set of coordinates for the 'secondary'
proctor2011['RA2'] = proctor2011['RA1']
proctor2011['Dec2'] = proctor2011['Dec1']

proctor2011['RA2_deg'] = proctor2011['RA1_deg']
proctor2011['Dec2_deg'] = proctor2011['Dec1_deg']

# Adding details about the coordinates
proctor2011['Equinox'] = "J2000"
proctor2011['Coordinate_waveband'] = "Radio"
proctor2011['Coordinate_Source'] = "FIRST"

proctor2011['System Type'] = 'Recoil Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
proctor2011['Brightness1'] = -100
proctor2011['Brightness_band1'] = -100
proctor2011['Brightness_type1'] = -100

proctor2011['Brightness2'] = -100
proctor2011['Brightness_band2'] = -100
proctor2011['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#proctor2011['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#proctor2011['Sep(kpc)'] = proctor2011['Sep']*((cosmo.arcsec_per_kpc_proper(proctor2011['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#proctor2011['delta_z'] = proctor2011['z1']-proctor2011['z2']
#proctor2011['dV'] = (2.99e+5)*((1+proctor2011['z1'])**2 - (1+proctor2011['z2'])**2)/((1+proctor2011['z1'])**2+(1+proctor2011['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
proctor2011['Selection Method'] = "Recoil Candidate / X-Shaped Radio Sources" #DPSELs
proctor2011['Confirmation Method'] = "-99"
proctor2011['Paper(s)'] = "Proctor+2011"
proctor2011['BibCode(s)'] = "2011ApJS..194...31P"
proctor2011['DOI(s)'] = "https://doi.org/10.1088/0067-0049/194/2/31"

## And dropping any columns that we don't need....
#yang2019.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#yang2019 = proctor2011[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#yang2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# These two objects are flagged as having double-peaked emission lines
#J0818+1508 J1554+3811


proctor2011


,---,FCG,RAh,RAm,RAs,DE-,DEd,DEm,DEs,Size,u_Size,Type,RAc.h,RAc.m,RAc.s,DEc.-,DEc.d,DEc.m,DEc.s,Group,Group2,Group3,Group4,Com,Name1,Name2,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,FCG,J002828.2-002634,0,28,28.288,-,0,26,34.65,2.74,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,3514,<NA>,<NA>,<NA>,mg,J002828.2-002634,J002828.2-002634,0:28:28.288,-0:26:34.65,7.117867,-0.442958,0:28:28.288,-0:26:34.65,7.117867,-0.442958,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
1,FCG,J004938.8+005947,0,49,38.814,+,0,59,47.20,0.57,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,6225,<NA>,<NA>,<NA>,rc sz dl? int,J004938.8+005947,J004938.8+005947,0:49:38.814,+0:59:47.2,12.411725,0.996444,0:49:38.814,+0:59:47.2,12.411725,0.996444,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
2,FCG,J011527.5-000000,1,15,27.574,-,0,0,0.98,1.38,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,9304,<NA>,<NA>,<NA>,int,J011527.5-000000,J011527.5-000000,1:15:27.574,-0:0:0.98,18.864892,-0.000272,1:15:27.574,-0:0:0.98,18.864892,-0.000272,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
3,FCG,J015043.7-081850,1,50,43.762,-,8,18,50.06,0.42,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,13599,<NA>,<NA>,<NA>,butterfly - (J2000) 01 50 43.550 -08 18 49.28,J015043.7-081850,J015043.7-081850,1:50:43.762,-8:18:50.06,27.682342,-8.313906,1:50:43.762,-8:18:50.06,27.682342,-8.313906,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
4,FCG,J015757.1-020625,1,57,57.184,-,2,6,25.20,1.26,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,14462,<NA>,<NA>,<NA>,t atypical,J015757.1-020625,J015757.1-020625,1:57:57.184,-2:6:25.2,29.488267,-2.107000,1:57:57.184,-2:6:25.2,29.488267,-2.107000,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
5,FCG,J021216.6-045038,2,12,16.648,-,4,50,38.12,1.46,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,17506,<NA>,<NA>,<NA>,tb?,J021216.6-045038,J021216.6-045038,2:12:16.648,-4:50:38.12,33.069367,-4.843922,2:12:16.648,-4:50:38.12,33.069367,-4.843922,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
6,FCG,J031937.6-020248,3,19,37.614,-,2,2,48.22,0.70,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,25536,<NA>,<NA>,<NA>,NaN,J031937.6-020248,J031937.6-020248,3:19:37.614,-2:2:48.22,49.906725,-2.046728,3:19:37.614,-2:2:48.22,49.906725,-2.046728,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
7,FCG,J072307.1+261822,7,23,7.138,+,26,18,22.86,0.77,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,35616,<NA>,<NA>,<NA>,NaN,J072307.1+261822,J072307.1+261822,7:23:7.138,+26:18:22.86,110.779742,26.306350,7:23:7.138,+26:18:22.86,110.779742,26.306350,J2000,Radio,FIRST,Recoil Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Proctor+2011,2011ApJS..194...31P,https://doi.org/10.1088/0067-0049/194/2/31
8,FCG,J072532.4+583528,7,25,32.410,+,58,35,28.95,1.03,NaN,NaN,<NA>,<NA>,NaN,NaN,<NA>,<NA>,NaN,37067,<NA>,<NA>,<NA>,NaN,J072532.4+583528,J07253

In [34]:
# Here we're adding in the information from Roberts+2018

roberts2018 = (Table.read('Tables/Roberts2018/table1.dat', readme = 'Tables/Roberts2018/ReadMe', format='ascii.cds')).to_pandas()
#Table 2 contains information on the VLA imaging
#Table 4 includes peak intensity information and figure tags

roberts_objs = roberts2018['[C2007]'].tolist()

for index, row in cheung_master.iterrows():
    if row['Name1'] in roberts_objs:
        #print('True')
        cheung_master.at[index, 'Paper(s)'] += ' ; Roberts+2018'
        cheung_master.at[index, 'BibCode(s)'] += ' ; 2018ApJ...852...47R' 
        cheung_master.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aa9c49'

cheung_master

,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Sep(kpc),delta_z,dV,Paper(s),BibCode(s),DOI(s),Notes
0,Recoil Candidate,4C 12.03,4C 12.03,Wing or X-Shaped Radio Source,-99,0.156000,-100,0.156000,-100,00:09:52.6115865762,+12:44:04.664804414,2.469215,12.734629,00:09:52.6115865762,+12:44:04.664804414,2.469215,12.734629,J2000,Optical,Gaia,17.80,B (Simbad),mag (Check Simbad),17.80,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Perryman+1984 ; Merritt+2002,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Coordinates from Simbad. Redshift from Perryman+.
1,Recoil Candidate,NGC 326,NGC 326,Wing or X-Shaped Radio Source,-99,0.047700,-100,0.047700,-100,00:58:22.637,+26:51:58.61,14.594321,26.866281,00:58:22.637,+26:51:58.61,14.594321,26.866281,J2000,Infrared,2MASS,13.00,V (Simbad),mag (Check Simbad),13.00,V (Simbad),mag (Check Simbad),0,0,0.0,0.0,Werner+1999 ; Merritt+2002,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Double galaxy. Redshift from Wener+. Coordinat...
2,Recoil Candidate,3C 52,3C 52,Wing or X-Shaped Radio Source,-99,0.290000,-100,0.290000,-100,01:48:28.884,+53:32:28.19,27.120350,53.541164,01:48:28.884,+53:32:28.19,27.120350,53.541164,J2000,Infrared,2MASS,18.50,B (Simbad),mag (Check Simbad),18.50,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Spinrad+1985 ; Merritt+2002,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Redshift from Ned and found in paper by Cheung...
3,Recoil Candidate,3C 63,3C 63,Wing or X-Shaped Radio Source,-99,0.175000,-100,0.175000,-100,02:20:54.3,-01:56:51,35.226250,-1.947500,02:20:54.3,-01:56:51,35.226250,-1.947500,J2000,Optical,NaN,18.50,B (Simbad),mag (Check Simbad),18.50,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Smith+1980 ; Cheung+2009,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,Redshift from paper by Cheung+2009. COordinate...
4,Recoil Candidate,3C 136.1,3C 136.1,Wing or X-Shaped Radio Source,-99,0.064000,-100,0.064000,-100,05:16:03.133,+24:58:25.47,79.013054,24.973742,05:16:03.133,+24:58:25.47,79.013054,24.973742,J2000,Infrared,2MASS,17.00,B (Simbad),mag (Check Simbad),17.00,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Spinrad+1985 ; Merritt+2002,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Coordinates from Simbad. Redshift from Hewitt+...
5,Recoil Candidate,3C 192,3C 192,Wing or X-Shaped Radio Source,-99,0.059680,-100,0.059680,-100,08:05:35.0057673498,+24:09:50.334614040,121.395857,24.163982,08:05:35.0057673498,+24:09:50.334614040,121.395857,24.163982,J2000,Optical,Gaia,15.46,B (Simbad),mag (Check Simbad),15.46,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Sandage+1966,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,Small wings. Coordinates and redshift from Sim...
6,Recoil Candidate,4C 32.25,4C 32.25,Wing or X-Shaped Radio Source,-99,0.051200,-100,0.051200,-100,08:31:27.521,+32:19:27.06,127.864671,32.324183,08:31:27.521,+32:19:27.06,127.864671,32.324183,J2000,Infrared,2MASS,16.00,B (Simbad),mag (Check Simbad),16.00,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,de Ruiter+1986 ; Merritt+2002,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Also known as B2 0828+32. Coordinates and reds...
7,Recoil Candidate,3C 223.1,3C 223.1,Wing or X-Shaped Radio Source,-99,0.107500,-100,0.107500,-100,09:41:24.0272828657,+39:44:41.806341013,145.350114,39.744946,09:41:24.0272828657,+39:44:41.806341013,145.350114,39.744946,J2000,Optical,Gaia,16.40,V (Simbad),mag (Check Simbad),16.40,V (Simbad),mag (Check Simbad),0,0,0.0,0.0,Sandage+1966 ; Dennett-Thorpe+2002 ; Merritt+2002,; 2002MNRAS.330..609D ; 2002Sci...297.1310M ;...,; https://doi.org/10.1046/j.1365-871

In [36]:
# Here we're adding in the information from Saripalli+2018
# Table 1 has redshift info, optical classes, XRG classification schemes, and size of radio extent
# Table 2 has information on hosts, emission lines, and radio power

# Here we're adding in the information from Roberts+2018

saripalli2018 = (Table.read('Tables/Saripalli2018/table1.dat', readme = 'Tables/Saripalli2018/ReadMe', format='ascii.cds')).to_pandas()
#Table 2 contains information on the VLA imaging
#Table 4 includes peak intensity information and figure tags

sar_objs = saripalli2018['[C2007]'].tolist()

for index, row in cheung_master.iterrows():
    if row['Name1'] in sar_objs:
        #print('True')
        cheung_master.at[index, 'Paper(s)'] += ' ; Saripalli+2018'
        cheung_master.at[index, 'BibCode(s)'] += ' ; 2018ApJ...852...48S' 
        cheung_master.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aa9c4b'

sarobjs = ['J0033−0149','J0049+0059','J0143−0119','J0821+2922','J1111+4050','J1128+1919','J1201−0703','J1210+1121','J1227+2155','J1330−0206','J1339−0016','J1522+4527','J2226+0125']      
for index, row in cheung_master.iterrows():
    if row['Name1'] in sar_objs:
        #print('True')
        cheung_master.at[index, 'Paper(s)'] += ' ; Saripalli+2018'
        cheung_master.at[index, 'BibCode(s)'] += ' ; 2018ApJ...852...48S' 
        cheung_master.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-4357/aa9c4b'
        cheung_master.at[index, 'Notes'] += ' Saripalli+ reject this object for reasons listed in their Table 3.'
            
cheung_master

,System Type,Name1,Name2,Selection Method,Confirmation Method,z1,z1_type,z2,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Sep,Sep(kpc),delta_z,dV,Paper(s),BibCode(s),DOI(s),Notes
0,Recoil Candidate,4C 12.03,4C 12.03,Wing or X-Shaped Radio Source,-99,0.156000,-100,0.156000,-100,00:09:52.6115865762,+12:44:04.664804414,2.469215,12.734629,00:09:52.6115865762,+12:44:04.664804414,2.469215,12.734629,J2000,Optical,Gaia,17.80,B (Simbad),mag (Check Simbad),17.80,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Perryman+1984 ; Merritt+2002,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Coordinates from Simbad. Redshift from Perryman+.
1,Recoil Candidate,NGC 326,NGC 326,Wing or X-Shaped Radio Source,-99,0.047700,-100,0.047700,-100,00:58:22.637,+26:51:58.61,14.594321,26.866281,00:58:22.637,+26:51:58.61,14.594321,26.866281,J2000,Infrared,2MASS,13.00,V (Simbad),mag (Check Simbad),13.00,V (Simbad),mag (Check Simbad),0,0,0.0,0.0,Werner+1999 ; Merritt+2002,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Double galaxy. Redshift from Wener+. Coordinat...
2,Recoil Candidate,3C 52,3C 52,Wing or X-Shaped Radio Source,-99,0.290000,-100,0.290000,-100,01:48:28.884,+53:32:28.19,27.120350,53.541164,01:48:28.884,+53:32:28.19,27.120350,53.541164,J2000,Infrared,2MASS,18.50,B (Simbad),mag (Check Simbad),18.50,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Spinrad+1985 ; Merritt+2002,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Redshift from Ned and found in paper by Cheung...
3,Recoil Candidate,3C 63,3C 63,Wing or X-Shaped Radio Source,-99,0.175000,-100,0.175000,-100,02:20:54.3,-01:56:51,35.226250,-1.947500,02:20:54.3,-01:56:51,35.226250,-1.947500,J2000,Optical,NaN,18.50,B (Simbad),mag (Check Simbad),18.50,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Smith+1980 ; Cheung+2009,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,Redshift from paper by Cheung+2009. COordinate...
4,Recoil Candidate,3C 136.1,3C 136.1,Wing or X-Shaped Radio Source,-99,0.064000,-100,0.064000,-100,05:16:03.133,+24:58:25.47,79.013054,24.973742,05:16:03.133,+24:58:25.47,79.013054,24.973742,J2000,Infrared,2MASS,17.00,B (Simbad),mag (Check Simbad),17.00,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Spinrad+1985 ; Merritt+2002,; 2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Coordinates from Simbad. Redshift from Hewitt+...
5,Recoil Candidate,3C 192,3C 192,Wing or X-Shaped Radio Source,-99,0.059680,-100,0.059680,-100,08:05:35.0057673498,+24:09:50.334614040,121.395857,24.163982,08:05:35.0057673498,+24:09:50.334614040,121.395857,24.163982,J2000,Optical,Gaia,15.46,B (Simbad),mag (Check Simbad),15.46,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,Sandage+1966,nan ; 2007AJ....133.2097C,nan ; https://doi.org/10.1086/513095,Small wings. Coordinates and redshift from Sim...
6,Recoil Candidate,4C 32.25,4C 32.25,Wing or X-Shaped Radio Source,-99,0.051200,-100,0.051200,-100,08:31:27.521,+32:19:27.06,127.864671,32.324183,08:31:27.521,+32:19:27.06,127.864671,32.324183,J2000,Infrared,2MASS,16.00,B (Simbad),mag (Check Simbad),16.00,B (Simbad),mag (Check Simbad),0,0,0.0,0.0,de Ruiter+1986 ; Merritt+2002,2002Sci...297.1310M ; 2007AJ....133.2097C,; https://doi.org/10.1126/science.1074688 ; h...,Also known as B2 0828+32. Coordinates and reds...
7,Recoil Candidate,3C 223.1,3C 223.1,Wing or X-Shaped Radio Source,-99,0.107500,-100,0.107500,-100,09:41:24.0272828657,+39:44:41.806341013,145.350114,39.744946,09:41:24.0272828657,+39:44:41.806341013,145.350114,39.744946,J2000,Optical,Gaia,16.40,V (Simbad),mag (Check Simbad),16.40,V (Simbad),mag (Check Simbad),0,0,0.0,0.0,Sandage+1966 ; Dennett-Thorpe+2002 ; Merritt+2002,; 2002MNRAS.330..609D ; 2002Sci...297.1310M ;...,; https://doi.org/10.1046/j.1365-871

In [38]:
#saripalli2018

In [8]:
# Now loading in the new X-shaped radio source from Yang+2019

# GO INTO YANG+ AND FLAG OBJECTS THEY QUOTE AS BEING DISCOVERED PREVIOUSLY!


yang2019t1 = (Table.read('Tables/Yang2019/table2.dat', readme = 'Tables/Yang2019/ReadMe', format='ascii.cds')).to_pandas()
yang2019t2 = (Table.read('Tables/Yang2019/table3.dat', readme = 'Tables/Yang2019/ReadMe', format='ascii.cds')).to_pandas()

yang2019t1['Name1'] = yang2019t1['SName']
yang2019t1['Name2'] = yang2019t1['SName']
yang2019t1['z1'] = yang2019t1['z']
yang2019t1['z2'] = yang2019t1['z']
yang2019t1['z1_type'] = yang2019t1['n_z']
yang2019t1['z2_type'] = yang2019t1['n_z']

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(yang2019,yang2019t1['Designation'])

yang2019t1['RA1'] = yang2019t1['RAh'].astype(str) + ':' + yang2019t1['RAm'].astype(str) + ':' + yang2019t1['RAs'].astype(str)
yang2019t1['Dec1'] = yang2019t1['DE-'].astype(str) + yang2019t1['DEd'].astype(str) + ':' + yang2019t1['DEm'].astype(str) + ':' + yang2019t1['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = yang2019t1['RA1'], dec = yang2019t1['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

yang2019t1['RA1_deg'] = coordconvert.ra.degree
yang2019t1['Dec1_deg'] = coordconvert.dec.degree

#yang2019t1['Coordinates'] = yang2019t1['SDSSID']#.str.slice(start=1) # Stripping the J
#yang2019t1['RA_test'] = yang2019t1['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
#yang2019t1['Dec_test'] = yang2019t1['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
#yang2019t1['RA'] = yang2019t1['RA_test'].str.slice(start=0, stop=2)+":"+yang2019t1['RA_test'].str.slice(start=2, stop=4)+":"+yang2019t1['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#yang2019t1['Dec'] = yang2019t1['Dec_test'].str.slice(start=0, stop=3)+":"+yang2019t1['Dec_test'].str.slice(start=3, stop=5)+":"+yang2019t1['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#yang2019.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Adding in a second set of coordinates for the 'secondary'
yang2019t1['RA2'] = yang2019t1['RA1']
yang2019t1['Dec2'] = yang2019t1['Dec1']

yang2019t1['RA2_deg'] = yang2019t1['RA1_deg']
yang2019t1['Dec2_deg'] = yang2019t1['Dec1_deg']

# Adding details about the coordinates
yang2019t1['Equinox'] = "J2000"
yang2019t1['Coordinate_waveband'] = "Optical"
yang2019t1['Coordinate_Source'] = "SDSS"

yang2019t1['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
yang2019t1['Brightness1'] = -100
yang2019t1['Brightness_band1'] = -100
yang2019t1['Brightness_type1'] = -100

yang2019t1['Brightness2'] = -100
yang2019t1['Brightness_band2'] = -100
yang2019t1['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#yang2019t1['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#yang2019t1['Sep(kpc)'] = yang2019t1['Sep']*((cosmo.arcsec_per_kpc_proper(yang2019t1['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#yang2019t1['delta_z'] = yang2019t1['z1']-yang2019t1['z2']
#yang2019t1['dV'] = (2.99e+5)*((1+yang2019t1['z1'])**2 - (1+yang2019t1['z2'])**2)/((1+yang2019t1['z1'])**2+(1+yang2019t1['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
yang2019t1['Selection Method'] = "Recoil Candidate / X-Shaped Radio Sources" #DPSELs
yang2019t1['Confirmation Method'] = "-99"
yang2019t1['Paper(s)'] = "Yang+2019 ; Joshi+2019"
yang2019t1['BibCode(s)'] = "2019ApJS..245...17Y ; 2019ApJ...887..266J"
yang2019t1['DOI(s)'] = "https://doi.org/10.3847/1538-4365/ab4811 ; https://doi.org/ 10.3847/1538-4357/ab536f"

## And dropping any columns that we don't need....
#yang2019.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#yang2019 = yang2019t1[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#yang2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# These two objects are flagged as having double-peaked emission lines
#J0818+1508 J1554+3811

yang2019t1


,SName,f_SName,RAh,RAm,RAs,DE-,DEd,DEm,DEs,z,n_z,rmag,RMag,S0.15,S1.4,S5,r_S5,alph1,alph2,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,J0004+1248,NaN,0,4,50.27,+,12,48,39.5,0.143,SPEC,17.44,-21.37,NaN,0.107,0.407,NRAO,NaN,1.07,J0004+1248,J0004+1248,0.143,0.143,SPEC,SPEC,0:4:50.27,+12:48:39.5,1.209458,12.810972,0:4:50.27,+12:48:39.5,1.209458,12.810972,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
1,J0028-0026,a,0,28,28.94,-,0,26,24.6,0.287,PHOT,18.96,-21.52,NaN,NaN,0.390,NRAO,NaN,NaN,J0028-0026,J0028-0026,0.287,0.287,PHOT,PHOT,0:28:28.94,-0:26:24.6,7.120583,-0.440167,0:28:28.94,-0:26:24.6,7.120583,-0.440167,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
2,J0030+1121,NaN,0,30,23.86,+,11,21,12.5,0.449,SPEC,20.16,-22.06,0.515,0.114,0.136,NRAO,-0.67,0.14,J0030+1121,J0030+1121,0.449,0.449,SPEC,SPEC,0:30:23.86,+11:21:12.5,7.599417,11.353472,0:30:23.86,+11:21:12.5,7.599417,11.353472,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
3,J0121+0051,NaN,1,21,1.23,+,0,51,0.3,0.238,SPEC,17.80,-22.16,1.124,0.267,0.105,NRAO,-0.64,-0.75,J0121+0051,J0121+0051,0.238,0.238,SPEC,SPEC,1:21:1.23,+0:51:0.3,20.255125,0.850083,1:21:1.23,+0:51:0.3,20.255125,0.850083,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
4,J0216+0244,NaN,2,16,35.79,+,2,44,0.9,0.184,PHOT,17.58,-21.70,2.650,0.300,0.248,PMN,-0.97,-0.15,J0216+0244,J0216+0244,0.184,0.184,PHOT,PHOT,2:16:35.79,+2:44:0.9,34.149125,2.733583,2:16:35.79,+2:44:0.9,34.149125,2.733583,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
5,J0319-0202,a,3,19,37.58,-,2,2,48.7,NaN,NaN,NaN,NaN,0.849,0.212,0.080,NRAO,-0.62,-0.78,J0319-0202,J0319-0202,NaN,NaN,NaN,NaN,3:19:37.58,-2:2:48.7,49.906583,-2.046861,3:19:37.58,-2:2:48.7,49.906583,-2.046861,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
6,J0710+3546,NaN,7,10,31.14,+,35,46,49.8,NaN,NaN,NaN,NaN,NaN,0.073,0.030,NRAO,NaN,-0.71,J0710+3546,J0710+3546,NaN,NaN,NaN,NaN,7:10:31.14,+35:46:49.8,107.629750,35.780500,7:10:31.14,+35:46:49.8,107.629750,35.780500,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
7,J0715+4910,NaN,7,15,10.12,+,49,10,53.2,NaN,NaN,NaN,NaN,0.513,0.084,0.044,NRAO,-0.81,-0.52,J0715+4910,J0715+4910,NaN,NaN,NaN,NaN,7:15:10.12,+49:10:53.2,108.792167,49.181444,7:15:10.12,+49:10:53.2,108.792167,49.181444,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019 ; Joshi+2019,2019ApJS..245...17Y ; 2019ApJ...887..266J,https://doi.org/10.3847/1538-4365/ab4811 ; htt...
8,J0720+4037,NaN,7,2

In [9]:
# Now loading in the new X-shaped radio source from Yang+2019
yang2019t2['Name1'] = yang2019t2['SName']
yang2019t2['Name2'] = yang2019t2['SName']
yang2019t2['z1'] = yang2019t2['z']
yang2019t2['z2'] = yang2019t2['z']
yang2019t2['z1_type'] = yang2019t2['n_z']
yang2019t2['z2_type'] = yang2019t2['n_z']

# Now converting the naming convention to RA and Dec and adding some informative columns
#name_to_coords(yang2019,yang2019t2['Designation'])

yang2019t2['RA1'] = yang2019t2['RAh'].astype(str) + ':' + yang2019t2['RAm'].astype(str) + ':' + yang2019t2['RAs'].astype(str)
yang2019t2['Dec1'] = yang2019t2['DE-'].astype(str) + yang2019t2['DEd'].astype(str) + ':' + yang2019t2['DEm'].astype(str) + ':' + yang2019t2['DEs'].astype(str)

# And now converting to get the coordinates in degrees rather than sexagesimal...
coordconvert = SkyCoord(ra = yang2019t2['RA1'], dec = yang2019t2['Dec1'], frame='icrs', unit = (u.hourangle, u.deg))

yang2019t2['RA1_deg'] = coordconvert.ra.degree
yang2019t2['Dec1_deg'] = coordconvert.dec.degree

#yang2019t2['Coordinates'] = yang2019t2['SDSSID']#.str.slice(start=1) # Stripping the J
#yang2019t2['RA_test'] = yang2019t2['Coordinates'].str.slice(start=1, stop=10) # Stripping the DEC parts 
#yang2019t2['Dec_test'] = yang2019t2['Coordinates'].str.slice(start=10, stop=19) # Stripping the RA parts
#yang2019t2['RA'] = yang2019t2['RA_test'].str.slice(start=0, stop=2)+":"+yang2019t2['RA_test'].str.slice(start=2, stop=4)+":"+yang2019t2['RA_test'].str.slice(start=4, stop=9) # Putting together the RA coordinates separated by colons
#yang2019t2['Dec'] = yang2019t2['Dec_test'].str.slice(start=0, stop=3)+":"+yang2019t2['Dec_test'].str.slice(start=3, stop=5)+":"+yang2019t2['Dec_test'].str.slice(start=5, stop=10) # Putting together the Dec coodinates separated by colons
#yang2019.drop(columns=['Coordinates','RA_test','Dec_test'], inplace=True)

# Adding in a second set of coordinates for the 'secondary'
yang2019t2['RA2'] = yang2019t2['RA1']
yang2019t2['Dec2'] = yang2019t2['Dec1']

yang2019t2['RA2_deg'] = yang2019t2['RA1_deg']
yang2019t2['Dec2_deg'] = yang2019t2['Dec1_deg']

# Adding details about the coordinates
yang2019t2['Equinox'] = "J2000"
yang2019t2['Coordinate_waveband'] = "Optical"
yang2019t2['Coordinate_Source'] = "SDSS"

yang2019t2['System Type'] = 'Binary AGN Candidate'

# Adding in some columns that we'll population via a Simbad or Ned search later
yang2019t2['Brightness1'] = -100
yang2019t2['Brightness_band1'] = -100
yang2019t2['Brightness_type1'] = -100

yang2019t2['Brightness2'] = -100
yang2019t2['Brightness_band2'] = -100
yang2019t2['Brightness_type2'] = -100

# Adding in a column to denote the system separation as '-1' which I will take in this case to mean that it is \
# of order ~1 kpc or less, but is not currently determined.
#yang2019t2['Sep'] = 3 # arcseconds
# Since these are candidates and we do not have a measure of separation, we'll use the 3'' diameter of the SDSS \
# fiber as an upper limit


#yang2019t2['Sep(kpc)'] = yang2019t2['Sep']*((cosmo.arcsec_per_kpc_proper(yang2019t2['z']))**(-1))


# For the projected separation, we'll use the upper limit of 3'' to calculate an upper limit in units of kpc
#yang2019t2['delta_z'] = yang2019t2['z1']-yang2019t2['z2']
#yang2019t2['dV'] = (2.99e+5)*((1+yang2019t2['z1'])**2 - (1+yang2019t2['z2'])**2)/((1+yang2019t2['z1'])**2+(1+yang2019t2['z2'])**2)
# dV will be zero until we include follow-up observations that show separate redshifts

# Adding information about the paper and the selection method
yang2019t2['Selection Method'] = "Recoil Candidate / X-Shaped Radio Sources" #DPSELs
yang2019t2['Confirmation Method'] = "-99"
yang2019t2['Paper(s)'] = "Yang+2019"
yang2019t2['BibCode(s)'] = "2019ApJS..245...17Y"
yang2019t2['DOI(s)'] = "https://doi.org/10.3847/1538-4365/ab4811"

## And dropping any columns that we don't need....
#yang2019.drop(labels=['SDSS','f_SDSS','Vel','logL','Type','Q','zr','zh'],\
#              axis=1, inplace=True)

## Rearranging the columns and renaming columns now...
#yang2019 = yang2019t2[['System Type','Name','Name2','Selection Method','Confirmation Method','z','z1_type','z2',\
#                    'z2_type','RA', 'Dec', 'RA_deg','Dec_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']]
#yang2019.columns=['System Type','Name1','Name2','Selection Method','Confirmation Method','z1','z1_type','z2',\
#                    'z2_type','RA1', 'Dec1', 'RA1_deg','Dec1_deg','RA2','Dec2','RA2_deg','Dec2_deg',\
#                    'Equinox','Coordinate_waveband','Coordinate_Source','Brightness1','Brightness_band1',\
#                    'Brightness_type1','Brightness2','Brightness_band2','Brightness_type2','Sep','Sep(kpc)',\
#                    'delta_z','dV','Paper(s)','BibCode(s)','DOI(s)']
#

# this is flagged as having double-peaked emission lines:
#J1247+1948


yang2019t2

,SName,f_SName,RAh,RAm,RAs,DE-,DEd,DEm,DEs,z,n_z,rmag,RMag,S0.15,S1.4,S5,r_S5,alph1,alph2,Name1,Name2,z1,z2,z1_type,z2_type,RA1,Dec1,RA1_deg,Dec1_deg,RA2,Dec2,RA2_deg,Dec2_deg,Equinox,Coordinate_waveband,Coordinate_Source,System Type,Brightness1,Brightness_band1,Brightness_type1,Brightness2,Brightness_band2,Brightness_type2,Selection Method,Confirmation Method,Paper(s),BibCode(s),DOI(s)
0,J0009+0457,NaN,0,9,18.92,+,4,57,53.2,0.545,PHOT,21.20,-21.55,1.402,0.320,0.094,NRAO,NaN,NaN,J0009+0457,J0009+0457,0.545,0.545,PHOT,PHOT,0:9:18.92,+4:57:53.2,2.328833,4.964778,0:9:18.92,+4:57:53.2,2.328833,4.964778,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
1,J0028-0428,NaN,0,28,15.26,-,4,28,55.3,0.267,SPEC,17.51,-22.84,0.484,0.092,0.063,PMN,NaN,NaN,J0028-0428,J0028-0428,0.267,0.267,SPEC,SPEC,0:28:15.26,-4:28:55.3,7.063583,-4.482028,0:28:15.26,-4:28:55.3,7.063583,-4.482028,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
2,J0031-0421,NaN,0,31,41.91,-,4,21,56.0,0.485,SPEC,21.05,-21.56,0.133,0.066,NaN,NaN,NaN,NaN,J0031-0421,J0031-0421,0.485,0.485,SPEC,SPEC,0:31:41.91,-4:21:56.0,7.924625,-4.365556,0:31:41.91,-4:21:56.0,7.924625,-4.365556,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
3,J0049-0507,NaN,0,49,55.56,-,5,7,4.3,NaN,NaN,22.40,NaN,0.596,0.152,0.064,PMN,NaN,NaN,J0049-0507,J0049-0507,NaN,NaN,NaN,NaN,0:49:55.56,-5:7:4.3,12.481500,-5.117861,0:49:55.56,-5:7:4.3,12.481500,-5.117861,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
4,J0051+0728,NaN,0,51,27.90,+,7,28,56.5,0.512,PHOT,20.13,-22.71,4.058,0.506,0.153,NRAO,NaN,NaN,J0051+0728,J0051+0728,0.512,0.512,PHOT,PHOT,0:51:27.9,+7:28:56.5,12.866250,7.482361,0:51:27.9,+7:28:56.5,12.866250,7.482361,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
5,J0055-0829,NaN,0,55,51.49,-,8,29,53.8,0.705,SPEC,21.11,-23.04,1.495,0.337,0.112,PMN,NaN,NaN,J0055-0829,J0055-0829,0.705,0.705,SPEC,SPEC,0:55:51.49,-8:29:53.8,13.964542,-8.498278,0:55:51.49,-8:29:53.8,13.964542,-8.498278,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
6,J0125+0617,NaN,1,25,25.82,+,6,17,31.5,0.274,PHOT,18.81,-21.57,1.875,0.423,0.161,NRAO,NaN,NaN,J0125+0617,J0125+0617,0.274,0.274,PHOT,PHOT,1:25:25.82,+6:17:31.5,21.357583,6.292083,1:25:25.82,+6:17:31.5,21.357583,6.292083,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
7,J0131+0314,NaN,1,31,5.68,+,3,14,52.6,0.468,PHOT,20.48,-21.68,0.846,0.182,0.059,NRAO,NaN,NaN,J0131+0314,J0131+0314,0.468,0.468,PHOT,PHOT,1:31:5.68,+3:14:52.6,22.773667,3.247944,1:31:5.68,+3:14:52.6,22.773667,3.247944,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.3847/1538-4365/ab4811
8,J0138-0654,NaN,1,38,19.05,-,6,54,58.0,NaN,NaN,NaN,NaN,0.604,0.200,0.151,PMN,NaN,NaN,J0138-0654,J0138-0654,NaN,NaN,NaN,NaN,1:38:19.05,-6:54:58.0,24.579375,-6.916111,1:38:19.05,-6:54:58.0,24.579375,-6.916111,J2000,Optical,SDSS,Binary AGN Candidate,-100,-100,-100,-100,-100,-100,Recoil Candidate / X-Shaped Radio Sources,-99,Yang+2019,2019ApJS..245...17Y,https://doi.org/10.384

In [ ]:
# Adding in the bib information for the Lal+2007 targets, whch were drawn from the cheung+07 list

lal2019 = ['J0113+0106','J0115−0000','J0702+5002','J0859−0433','J0914+1715','J0917+0523','J0924+4233','J1055−0707','J1130+0058','J1218+1955','J1309−0012','J1339−0016','J1406−0154','J1430+5217','J1600+2058','J1606+0000']

for index, row in cheung_master.iterrows():
    if row['Name1'] in lal2019:
        #print('True')
        cheung_master.at[index, 'Paper(s)'] += ' ; Lal+2019'
        cheung_master.at[index, 'BibCode(s)'] += ' ; 2019AJ....157..195L' 
        cheung_master.at[index, 'DOI(s)'] += ' ; https://doi.org/10.3847/1538-3881/ab1419'
        cheung_master.at[index, 'Notes'] += ' Lal+2019 obtained additional radio imaging.'
            
cheung_master
